In [ ]:
# import sys
# !{sys.executable} -m pip install category_encoders

######################################################################  
######################################################################  
### --------------------- Setup
######################################################################  
######################################################################  

In [8]:
from pathlib import Path
import os
from joblib import Memory
from sklearn.datasets import load_svmlight_file
from scipy.sparse import csc_matrix
from scipy.special import expit
from scipy import sparse
from sklearn.preprocessing import normalize
from copy import copy
from sklearn import svm
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder

In [9]:
import pyarrow.feather as feather
import matplotlib.pyplot as plt
import plotnine as p9
import pandas as pd
import numpy as np

idcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/csc_variable_subsets/id_csc.feather"
cipcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/csc_variable_subsets/cip_csc.feather"
geoloccsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/csc_variable_subsets/geolocation_csc.feather"
instdemocsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/csc_variable_subsets/inst_demographic_csc.feather"
studdemocsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/csc_variable_subsets/stud_demographic_csc.feather"
pcipcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/csc_variable_subsets/pcip_csc.feather"
numcsc = "C:/Code/GITHUB/csc/Classification-of-Pell-Institutions/data/datasubsets/csc_variable_subsets/num_csc.feather"

full_df_path = "data/datasubsets/csc_variable_subsets/full_csc_frame.feather"

id_csc = feather.read_feather(idcsc)
cip_csc = feather.read_feather(cipcsc)
geolocation_csc = feather.read_feather(geoloccsc)
inst_demographic_csc = feather.read_feather(instdemocsc)
stud_demographic_csc = feather.read_feather(studdemocsc)
pcip_csc = feather.read_feather(pcipcsc)
num_csc = feather.read_feather(numcsc)

id_csc
cip_csc
geolocation_csc
inst_demographic_csc
stud_demographic_csc
pcip_csc
num_csc

month_dict = {"4":"Spring", "8":"Summer", "12": "Winter",
              "2": "Winter", "1": "Winter", "7": "Summer",
              "6": "Summer", "3": "Summer", "5": "Spring",
              "11": "Winter", "10": "Fall", "9": "Fall"}
		
id_csc["UNITID"] = id_csc["UNITID"].astype(object)
id_csc["FEDSCHCD"] = id_csc["FEDSCHCD"].astype(object)
id_csc["OPEID"] = id_csc["OPEID"].astype(object)
id_csc["OPEID6"] = id_csc["OPEID6"].astype(object)

geolocation_csc.INSTNM = geolocation_csc["INSTNM"].replace("[\&]", "and", regex = True)
geolocation_csc.INSTNM = geolocation_csc["INSTNM"].replace("[\"]", "", regex = True)
geolocation_csc.INSTNM = geolocation_csc["INSTNM"].replace("[\-]", " ", regex = True)
geolocation_csc.INSTNM = geolocation_csc.INSTNM.str.lower()
inst_demographic_csc.ACCREDAGENCY = inst_demographic_csc.ACCREDAGENCY.str.lower()

rand_state = 5991

In [10]:
# FUNCTIONS
def split_data(xx, yy, testsize = 1000):
    xtrain, xtest, y_train, y_test = train_test_split(xx,
                                                      yy,
                                                      test_size = testsize,
                                                      random_state = rand_state)
    xtrain, xvalid, y_train, y_valid = train_test_split(xtrain, y_train, 
                                                        test_size = testsize,
                                                        random_state = rand_state)

    y_train = np.array(y_train).reshape(-1, 1)
    y_test = np.array(y_test).reshape(-1, 1)
    y_valid = np.array(y_valid).reshape(-1, 1)

    print(xtrain.shape)
    print(y_train.shape)
    print(xtest.shape)
    print(y_test.shape)
    print(xvalid.shape)
    print(y_valid.shape)

    return xtrain, y_train, xtest, y_test, xvalid, y_valid 


In [11]:
ids = np.array(range(len(id_csc)))
id_csc["ids"] = ids
id_csc

,UNITID,FEDSCHCD,OPEID,OPEID6,INSTNM,PELLCAT,PCTPELL,SCHTYPE,ids
0,100654.0,1002,100200.0,1002.0,Alabama A & M University,1.0,0.7067,1.0,0
1,100663.0,1052,105200.0,1052.0,University of Alabama at Birmingham,0.0,0.3632,1.0,1
2,100690.0,16885,2503400.0,25034.0,Amridge University,1.0,0.7673,2.0,2
3,100706.0,1055,105500.0,1055.0,University of Alabama in Huntsville,0.0,0.2698,1.0,3
4,100724.0,1005,100500.0,1005.0,Alabama State University,1.0,0.7448,1.0,4
...,...,...,...,...,...,...,...,...,...
5874,493080.0,E40459,4144301.0,41443.0,Institute of Culinary Education,0.0,0.3269,3.0,5874
5875,493150.0,42386,4238601.0,42386.0,Gould's Academy,1.0,0.5556,3.0,5875
5876,493372.0,41624,4162401.0,41624.0,Toni & Guy Hairdressing Academy-Rio Rancho,1.0,0.5652,3.0,5876
5877,493424.0,E40460,3915304.0,39153.0,Career Quest Learning Center-Bay City Branch,1.0,0.5000,3.0,5877


In [12]:
target_variable = id_csc.loc[:,["ids", "UNITID", "PELLCAT"]]
target_variable.head(5)

,ids,UNITID,PELLCAT
0,0,100654.0,1.0
1,1,100663.0,0.0
2,2,100690.0,1.0
3,3,100706.0,0.0
4,4,100724.0,1.0


In [13]:
justids = id_csc.loc[:,["ids", "UNITID"]]
justids.head(5)


,ids,UNITID
0,0,100654.0
1,1,100663.0
2,2,100690.0
3,3,100706.0
4,4,100724.0


In [14]:
print("cip_csc")
b4_cip = cip_csc.shape
print("BEFORE:", b4_cip)
cip_csc = pd.merge(justids, 
                   cip_csc,
                   how = "left")
af_cip = cip_csc.shape
print("AFTER: ", af_cip)

print(" ")
print("geolocation_csc")
b4_geolocation_csc = geolocation_csc.shape
print("BEFORE:", b4_geolocation_csc)
geolocation_csc = pd.merge(justids, 
                           geolocation_csc,
                           how = "left")
af_geolocation_csc = geolocation_csc.shape
print("AFTER: ", af_geolocation_csc)

print(" ")
print("inst_demographic_csc")
b4_inst_demographic_csc = inst_demographic_csc.shape
print("BEFORE:", b4_inst_demographic_csc)
inst_demographic_csc = pd.merge(justids, 
                   inst_demographic_csc,
                   how = "left")
af_inst_demographic_csc = inst_demographic_csc.shape
print("AFTER: ", af_inst_demographic_csc)

print(" ")
print("stud_demographic_csc")
b4_stud_demographic_csc = stud_demographic_csc.shape
print("BEFORE:", b4_stud_demographic_csc)
stud_demographic_csc = pd.merge(justids, 
                   stud_demographic_csc,
                   how = "left")
af_stud_demographic_csc = stud_demographic_csc.shape
print("AFTER: ", af_stud_demographic_csc)

print(" ")
print("pcip_csc")
b4_pcip_csc = pcip_csc.shape
print("BEFORE:", b4_pcip_csc)
pcip_csc = pd.merge(justids, 
                   pcip_csc,
                   how = "left")
af_pcip_csc = pcip_csc.shape
print("AFTER: ", af_pcip_csc)

print(" ")
print("num_csc")
b4_num_csc = num_csc.shape
print("BEFORE:", b4_num_csc)
num_csc = pd.merge(justids, 
                   num_csc,
                   how = "left")
af_num_csc = num_csc.shape
print("AFTER: ", af_num_csc)

cip_csc
BEFORE: (5879, 192)
AFTER:  (5879, 193)
 
geolocation_csc
BEFORE: (5879, 13)
AFTER:  (5879, 14)
 
inst_demographic_csc
BEFORE: (5879, 14)
AFTER:  (5879, 15)
 
stud_demographic_csc
BEFORE: (5879, 12)
AFTER:  (5879, 13)
 
pcip_csc
BEFORE: (5879, 40)
AFTER:  (5879, 41)
 
num_csc
BEFORE: (5879, 163)
AFTER:  (5879, 164)


######################################################################  
######################################################################  
### --------------------- Imputing missing values
######################################################################  
######################################################################  

### Numeric Values

In [15]:
num_csc.drop(columns = ["ids", "UNITID", "INSTNM"], axis = 1).head(5)

,NUMBRANCH,PCTFLOAN,D_PCTPELL_PCTFLOAN,SCUGFFN,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,...,OMENRUP8_FTNFT,BBRR2_PP_UG_N,BBRR2_FED_UGNOCOMP_N,DBRR4_PP_UG_N,DBRR4_PP_UG_NUM,DBRR4_PP_UG_DEN,DBRR4_PP_UG_RT,TUITIONFEE_IN,TUITIONFEE_OUT,PLUS_DEBT_INST_MD
0,1.0,0.7503,5039.0,1288.0,4990.0,0.0186,0.9120,0.0088,0.0018,0.0022,...,0.1697,785.0,1858.0,668.0,10785863.0,8969946.0,1.202445,9744.0,18354.0,14838.0
1,1.0,0.5127,13134.0,2228.0,13186.0,0.5717,0.2553,0.0334,0.0633,0.0034,...,0.2114,597.0,2511.0,535.0,8537186.0,8543933.0,0.999210,8568.0,19704.0,16145.0
2,1.0,0.8962,318.0,5.0,351.0,0.2393,0.7151,0.0171,0.0057,0.0057,...,0.7027,NaN,247.0,NaN,NaN,NaN,NaN,6900.0,6900.0,NaN
3,1.0,0.4192,7090.0,1341.0,7458.0,0.7167,0.0969,0.0528,0.0381,0.0095,...,0.1667,237.0,1014.0,239.0,3494955.0,3914476.0,0.892828,10714.0,22362.0,13524.0
4,1.0,0.7845,4208.0,951.0,3903.0,0.0167,0.9352,0.0095,0.0041,0.0013,...,0.1759,799.0,1708.0,790.0,13008748.0,10302465.0,1.262683,11068.0,19396.0,15351.0


In [18]:
missing = pd.DataFrame(num_csc.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] == 0].shape)
missing[missing[0] != 0]

(7, 2)


,index,0
7,UGDS,1
8,UGDS_WHITE,1
9,UGDS_BLACK,1
10,UGDS_HISP,1
11,UGDS_ASIAN,1
...,...,...
159,DBRR4_PP_UG_DEN,2401
160,DBRR4_PP_UG_RT,2401
161,TUITIONFEE_IN,2427
162,TUITIONFEE_OUT,2427


In [ ]:
num_vars = num_csc.drop(columns = ["ids", "UNITID", "INSTNM"], axis = 1)

for x in list(num_vars):
    med = num_csc[x].median()
    num_csc[x].fillna(med, inplace = True)

missing = pd.DataFrame(num_csc.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] == 0].shape)
missing[missing[0] != 0]

#############################################################################################
#############################################################################################
#############################################################################################

### Categorical variables

In [ ]:
inst_demographic_csc["OPENADMP"].fillna(9, inplace=True)
inst_demographic_csc.ACCREDCODE = inst_demographic_csc.ACCREDCODE.replace("COE | WASCCS", "WASCCS", regex = True)
inst_demographic_csc.ACCREDCODE = inst_demographic_csc.ACCREDCODE.replace("WASCCS|WASCCS", "WASCCS", regex = True)
inst_demographic_csc.loc[inst_demographic_csc["ACCREDCODE"].isna(),["INSTNM", "ACCREDCODE", "CONTROL"]] = "NONE"
inst_demographic_csc.loc[inst_demographic_csc["ACCREDAGENCY"].isna(),["INSTNM", "ACCREDAGENCY", "CONTROL"]] = "NONE"
inst_demographic_csc["T4APPROVALDATE"].fillna("7/99/999", inplace=True)

stud_demographic_csc["HSI"].fillna(0, inplace=True)
stud_demographic_csc["HBCU"].fillna(0, inplace=True)
stud_demographic_csc["ANNHI"].fillna(0, inplace=True)
stud_demographic_csc["PBI"].fillna(0, inplace=True)
stud_demographic_csc["TRIBAL"].fillna(0, inplace=True)
stud_demographic_csc["NANTI"].fillna(0, inplace=True)
stud_demographic_csc["AANAPII"].fillna(0, inplace=True)

#### cip_csc

In [ ]:
missing = pd.DataFrame(cip_csc.isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] != 0].count()

In [ ]:
var_names = cip_csc.drop(columns = ["ids", "UNITID", "INSTNM"], axis = 1)

print("Original shape:", cip_csc.shape) # Original shape: (5879, 193)
print("Number of variables that will be encoded:", var_names.shape) # (5879, 190)

nber = 0
for x in list(var_names):
    enc = OneHotEncoder(handle_unknown="ignore")
    enc.fit(np.array(cip_csc[x]).reshape(-1, 1))
    num_category = len(np.array(enc.categories_).transpose())
    nber = nber + num_category

print("Number of variables to expect:", cip_csc.shape[1] + nber)

In [ ]:
for x in list(var_names):
    enc = OneHotEncoder(handle_unknown="ignore")

    enc.fit(np.array(cip_csc[x]).reshape(-1, 1))
    labels = np.array(cip_csc[x])
    encoded = enc.transform(labels.reshape(-1,1)).toarray()

    num_category = len(np.array(enc.categories_).transpose())
    var_name = np.array([(x + "_encoded{}").format(i) for i in range(num_category)])

    for i in range(len(var_name)):
        cip_csc[var_name[i]] = encoded[:, i]


In [ ]:
print(cip_csc.shape)

#######################################################################################

#### geolocation_csc

In [ ]:
missing = pd.DataFrame(geolocation_csc.isna().sum())
missing.reset_index(inplace=True)
missing[missing[0] != 0].count()
missing

In [ ]:
encoder = TargetEncoder()
encoder.fit(geolocation_csc["CITY"], id_csc["PELLCAT"])
geolocation_csc["CITY Encoded"] = encoder.transform(geolocation_csc["CITY"])
# geolocation_csc.loc[:,["STABBR", "CITY", "CITY Encoded"]]


encoder = TargetEncoder()
encoder.fit(geolocation_csc["INSTNM"], id_csc["PELLCAT"])
geolocation_csc["INSTNM Encoded"] = encoder.transform(geolocation_csc["INSTNM"])
# geolocation_csc.loc[:,["STABBR", "INSTNM", "INSTNM Encoded"]]

geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["ST_FIPS"].astype(int), prefix = "FIPS")],
                             axis = 1)

geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["REGION"].astype(int), prefix = "REGION")],
                             axis = 1)

geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["LOCALE"].astype(int), prefix = "LOC")],
                             axis = 1)
####
geolocation_csc.loc[geolocation_csc["CCBASIC"] == 99, "CCBASIC"] = 99
geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["CCBASIC"].astype(int), prefix = "CCB")],
                             axis = 1)
####
geolocation_csc.loc[geolocation_csc["CCSIZSET"] == 99, "CCSIZSET"] = 99
geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["CCSIZSET"].astype(int), prefix = "CCSS")],
                             axis = 1)
####
geolocation_csc.loc[geolocation_csc["CCUGPROF"] == 99, "CCUGPROF"] = 99
geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["CCUGPROF"].astype(int), prefix = "CCPROF")],
                             axis = 1)
####
geolocation_csc["ZIP5"] = geolocation_csc["ZIP"].str.slice(0, 5)
geolocation_csc["ZIP4"] = geolocation_csc["ZIP"].str.slice(0, 4)
geolocation_csc["ZIP3"] = geolocation_csc["ZIP"].str.slice(0, 3)
geolocation_csc["ZIP2"] = geolocation_csc["ZIP"].str.slice(0, 2)

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP2"], id_csc["PELLCAT"])
geolocation_csc["ZIP2 Encoded"] = encoder.transform(geolocation_csc["ZIP2"])
# geolocation_csc.loc[:,["STABBR", "ZIP2", "ZIP2 Encoded"]]

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP3"], id_csc["PELLCAT"])
geolocation_csc["ZIP3 Encoded"] = encoder.transform(geolocation_csc["ZIP3"])
# geolocation_csc.loc[:,["STABBR", "ZIP3", "ZIP3 Encoded"]]

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP4"], id_csc["PELLCAT"])
geolocation_csc["ZIP4 Encoded"] = encoder.transform(geolocation_csc["ZIP4"])
# geolocation_csc.loc[:,["STABBR", "ZIP4", "ZIP4 Encoded"]]

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP5"], id_csc["PELLCAT"])
geolocation_csc["ZIP5 Encoded"] = encoder.transform(geolocation_csc["ZIP5"])
# geolocation_csc.loc[:,["STABBR", "ZIP5", "ZIP5 Encoded"]]

In [ ]:
geolocation_csc.head(5)

#######################################################################################

#### inst_demographic_csc

In [ ]:
missing = pd.DataFrame(inst_demographic_csc.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] != 0].count())
missing[missing[0] != 0]

# ddd = inst_demographic_csc[(inst_demographic_csc["Season"].isna()) == True].loc[:, ["INSTNM", "Season", "T4_month"]]
# ddd["T4_month"].map(month_dict) 
# inst_demographic_csc["T4_month"] = inst_demographic_csc["T4_month"].replace("99", inst_demographic_csc["T4_month"].median())
# inst_demographic_csc["T4_month"]
# inst_demographic_csc["Season"] = inst_demographic_csc["T4_month"].map(month_dict) 

In [ ]:
inst_demographic_csc = pd.concat((inst_demographic_csc,
                                  pd.DataFrame(np.array(list(inst_demographic_csc["T4APPROVALDATE"].str.split("/"))),
                                               columns = ["T4_month", "T4_day", "T4_year"])),
                                  axis = 1)

inst_demographic_csc.loc[:, ["INSTNM", "T4APPROVALDATE", "T4_month", "T4_day", "T4_year"]]


inst_demographic_csc["Season"] = inst_demographic_csc["T4_month"].map(month_dict)                                 

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["MAIN"].astype(int),
                                  prefix = "MAIN")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["HCM2"].astype(int),
                                  prefix = "HCM2")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["OPEFLAG"].astype(int),
                                  prefix = "OPEFLAG")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["PREDDEG"].astype(int),
                                  prefix = "PREDDEG")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["HIGHDEG"].astype(int),
                                  prefix = "HIGHDEG")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["ICLEVEL"].astype(int),
                                  prefix = "ICLEVEL")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["CONTROL"],
                                  prefix = "CONTROL")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["CURROPER"].astype(int),
                                  prefix = "CURROPER")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["OPENADMP"].astype(int),
                                  prefix = "OPENADMP")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["ACCREDCODE"],
                                  prefix = "ACCRED")],
                                  axis = 1)

encoder = TargetEncoder()
encoder.fit(inst_demographic_csc["ACCREDAGENCY"], id_csc["PELLCAT"])
inst_demographic_csc["ACCREDAGENCY Encoded"] = encoder.transform(inst_demographic_csc["ACCREDAGENCY"])

encoder = TargetEncoder()
encoder.fit(inst_demographic_csc["Season"], id_csc["PELLCAT"])
inst_demographic_csc["Season Encoded"] = encoder.transform(inst_demographic_csc["Season"])

In [ ]:
inst_demographic_csc.head(5)

#######################################################################################

#### stud_demographic_csc

In [ ]:
missing = pd.DataFrame(stud_demographic_csc.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] != 0].count())
missing[missing[0] != 0]

In [ ]:
stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["HSI"],
                                  prefix = "HSI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["HBCU"],
                                  prefix = "HBCU")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["ANNHI"],
                                  prefix = "ANNHI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["PBI"],
                                  prefix = "PBI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["TRIBAL"],
                                  prefix = "TRIBAL")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["NANTI"],
                                  prefix = "NANTI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["AANAPII"],
                                  prefix = "AANAPII")],
                                  axis = 1)

#######################################################################################

#### pcip_csc

In [ ]:
hodl = pcip_csc.copy()
missing = pd.DataFrame(pcip_csc.isna().sum())
missing.reset_index(inplace=True)
# missing[missing[0] != 0].count()
missing.iloc[41:81]

In [ ]:
pcip_csc["PCIP01_desc"]=pd.cut(pcip_csc["PCIP01"], bins = [-1, 0, 0.125, 0.325, 0.525, 0.725, 0.925, 1], labels = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1])
pcip_csc["PCIP03_desc"]=pd.cut(pcip_csc["PCIP03"], bins = [-1, 0, 0.007, 0.0175, 0.028, 0.0385, 0.049, 1], labels = [0, 0.007, 0.0175, 0.028, 0.0385, 0.049, 1])
pcip_csc["PCIP04_desc"]=pd.cut(pcip_csc["PCIP04"], bins = [-1, 0, 0.005, 0.0125, 0.02, 0.0275, 0.035, 1], labels = [0, 0.005, 0.0125, 0.02, 0.0275, 0.035, 1])
pcip_csc["PCIP05_desc"]=pd.cut(pcip_csc["PCIP05"], bins = [-1, 0, 0.002, 0.0035, 0.005, 0.0065, 0.008, 1], labels = [0, 0.002, 0.0035, 0.005, 0.0065, 0.008, 1])
pcip_csc["PCIP09_desc"]=pd.cut(pcip_csc["PCIP09"], bins = [-1, 0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 1], labels = [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 1])
pcip_csc["PCIP10_desc"]=pd.cut(pcip_csc["PCIP10"], bins = [-1, 0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004, 1], labels = [0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004, 1])
pcip_csc["PCIP11_desc"]=pd.cut(pcip_csc["PCIP11"], bins = [-1, 0, 0.03, 0.05, 0.07, 0.09, 0.11, 1], labels = [0, 0.03, 0.05, 0.07, 0.09, 0.11, 1])
pcip_csc["PCIP12_desc"]=pd.cut(pcip_csc["PCIP12"], bins = [-1, 0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1], labels = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1])
pcip_csc["PCIP13_desc"]=pd.cut(pcip_csc["PCIP13"], bins = [-1, 0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475, 1], labels = [0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475, 1])
pcip_csc["PCIP14_desc"]=pd.cut(pcip_csc["PCIP14"], bins = [-1, 0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1], labels = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1])
pcip_csc["PCIP15_desc"]=pd.cut(pcip_csc["PCIP15"], bins = [-1, 0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1], labels = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1])
pcip_csc["PCIP16_desc"]=pd.cut(pcip_csc["PCIP16"], bins = [-1, 0, 0.001, 0.004, 0.007, 0.01, 1], labels = [0, 0.001, 0.004, 0.007, 0.01, 1])
pcip_csc["PCIP19_desc"]=pd.cut(pcip_csc["PCIP19"], bins = [-1, 0, 0.005, 0.035, 0.065, 0.095, 1], labels = [0, 0.005, 0.035, 0.065, 0.095, 1])
pcip_csc["PCIP22_desc"]=pd.cut(pcip_csc["PCIP22"], bins = [-1, 0, 0.01, 0.035, 0.065, 0.095, 1], labels = [0, 0.01, 0.035, 0.065, 0.095, 1])
pcip_csc["PCIP23_desc"]=pd.cut(pcip_csc["PCIP23"], bins = [-1, 0, 0.001, 0.012, 0.023, 0.034, 1], labels = [0, 0.001, 0.012, 0.023, 0.034, 1])
pcip_csc["PCIP24_desc"]=pd.cut(pcip_csc["PCIP24"], bins = [-1, 0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481, 1], labels = [0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481, 1])
pcip_csc["PCIP25_desc"]=pd.cut(pcip_csc["PCIP25"], bins = [-1, 0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135, 1], labels = [0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135, 1])
pcip_csc["PCIP26_desc"]=pd.cut(pcip_csc["PCIP26"], bins = [-1, 0, 0.01, 0.04, 0.07, 0.1, 0.13, 1], labels = [0, 0.01, 0.04, 0.07, 0.1, 0.13, 1])
pcip_csc["PCIP27_desc"]=pd.cut(pcip_csc["PCIP27"], bins = [-1, 0, 0.0025, 0.01, 0.0175, 0.025, 0.0325, 1], labels = [0, 0.0025, 0.01, 0.0175, 0.025, 0.0325, 1])
pcip_csc["PCIP30_desc"]=pd.cut(pcip_csc["PCIP30"], bins = [-1, 0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1], labels = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1])
pcip_csc["PCIP31_desc"]=pd.cut(pcip_csc["PCIP31"], bins = [-1, 0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1], labels = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1])
pcip_csc["PCIP38_desc"]=pd.cut(pcip_csc["PCIP38"], bins = [-1, 0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036, 1], labels = [0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036, 1])
pcip_csc["PCIP39_desc"]=pd.cut(pcip_csc["PCIP39"], bins = [-1, 0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275, 1], labels = [0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275, 1])
pcip_csc["PCIP40_desc"]=pd.cut(pcip_csc["PCIP40"], bins = [-1, 0, 0.001, 0.006, 0.011, 0.016, 0.021, 1], labels = [0, 0.001, 0.006, 0.011, 0.016, 0.021, 1])
pcip_csc["PCIP41_desc"]=pd.cut(pcip_csc["PCIP41"], bins = [-1, 0, 0.0005, 0.003, 0.0055, 0.008, 0.0105, 1], labels = [0, 0.0005, 0.003, 0.0055, 0.008, 0.0105, 1])
pcip_csc["PCIP42_desc"]=pd.cut(pcip_csc["PCIP42"], bins = [-1, 0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11, 1], labels = [0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11, 1])
pcip_csc["PCIP43_desc"]=pd.cut(pcip_csc["PCIP43"], bins = [-1, 0, 0.01, 0.03, 0.05, 0.07, 0.09, 1], labels = [0, 0.01, 0.03, 0.05, 0.07, 0.09, 1])
pcip_csc["PCIP44_desc"]=pd.cut(pcip_csc["PCIP44"], bins = [-1, 0, 0.005, 0.02, 0.035, 0.05, 0.065, 1], labels = [0, 0.005, 0.02, 0.035, 0.05, 0.065, 1])
pcip_csc["PCIP45_desc"]=pd.cut(pcip_csc["PCIP45"], bins = [-1, 0, 0.005, 0.02, 0.035, 0.05, 0.065, 1], labels = [0, 0.005, 0.02, 0.035, 0.05, 0.065, 1])
pcip_csc["PCIP46_desc"]=pd.cut(pcip_csc["PCIP46"], bins = [-1, 0, 0.0103, 0.037796, 0.125, 1], labels = [0, 0.0103, 0.037796, 0.125, 1])
pcip_csc["PCIP47_desc"]=pd.cut(pcip_csc["PCIP47"], bins = [-1, 0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615, 1], labels = [0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615, 1])
pcip_csc["PCIP48_desc"]=pd.cut(pcip_csc["PCIP48"], bins = [-1, 0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1], labels = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1])
pcip_csc["PCIP49_desc"]=pd.cut(pcip_csc["PCIP49"], bins = [-1, 0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1], labels = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1])
pcip_csc["PCIP50_desc"]=pd.cut(pcip_csc["PCIP50"], bins = [-1, 0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1], labels = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1])
pcip_csc["PCIP51_desc"]=pd.cut(pcip_csc["PCIP51"], bins = [-1, 0, 0.125, 0.325, 0.525, 0.725, 0.925, 1], labels = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1])
pcip_csc["PCIP52_desc"]=pd.cut(pcip_csc["PCIP52"], bins = [-1, 0, 0.125, 0.325, 0.525, 0.725, 0.925, 1], labels = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1])
pcip_csc["PCIP54_desc"]=pd.cut(pcip_csc["PCIP54"], bins = [-1, 0, 0.125, 0.325, 0.525, 0.725, 0.925, 1], labels = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1])

In [ ]:
# look around the new variables and fill any na's
# pcip_csc.loc[pcip_csc["PCIP01"] == 0, "PCIP01_desc"] = pcip_csc["PCIP01_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP03"] == 0, "PCIP03_desc"] = pcip_csc["PCIP03_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP04"] == 0, "PCIP04_desc"] = pcip_csc["PCIP04_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP05"] == 0, "PCIP05_desc"] = pcip_csc["PCIP05_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP09"] == 0, "PCIP09_desc"] = pcip_csc["PCIP09_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP10"] == 0, "PCIP10_desc"] = pcip_csc["PCIP10_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP11"] == 0, "PCIP11_desc"] = pcip_csc["PCIP11_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP12"] == 0, "PCIP12_desc"] = pcip_csc["PCIP12_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP13"] == 0, "PCIP13_desc"] = pcip_csc["PCIP13_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP14"] == 0, "PCIP14_desc"] = pcip_csc["PCIP14_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP15"] == 0, "PCIP15_desc"] = pcip_csc["PCIP15_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP16"] == 0, "PCIP16_desc"] = pcip_csc["PCIP16_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP19"] == 0, "PCIP19_desc"] = pcip_csc["PCIP19_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP22"] == 0, "PCIP22_desc"] = pcip_csc["PCIP22_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP23"] == 0, "PCIP23_desc"] = pcip_csc["PCIP23_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP24"] == 0, "PCIP24_desc"] = pcip_csc["PCIP24_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP25"] == 0, "PCIP25_desc"] = pcip_csc["PCIP25_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP26"] == 0, "PCIP26_desc"] = pcip_csc["PCIP26_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP27"] == 0, "PCIP27_desc"] = pcip_csc["PCIP27_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP30"] == 0, "PCIP30_desc"] = pcip_csc["PCIP30_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP31"] == 0, "PCIP31_desc"] = pcip_csc["PCIP31_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP38"] == 0, "PCIP38_desc"] = pcip_csc["PCIP38_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP39"] == 0, "PCIP39_desc"] = pcip_csc["PCIP39_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP40"] == 0, "PCIP40_desc"] = pcip_csc["PCIP40_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP41"] == 0, "PCIP41_desc"] = pcip_csc["PCIP41_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP42"] == 0, "PCIP42_desc"] = pcip_csc["PCIP42_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP43"] == 0, "PCIP43_desc"] = pcip_csc["PCIP43_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP44"] == 0, "PCIP44_desc"] = pcip_csc["PCIP44_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP45"] == 0, "PCIP45_desc"] = pcip_csc["PCIP45_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP46"] == 0, "PCIP46_desc"] = pcip_csc["PCIP46_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP47"] == 0, "PCIP47_desc"] = pcip_csc["PCIP47_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP48"] == 0, "PCIP48_desc"] = pcip_csc["PCIP48_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP49"] == 0, "PCIP49_desc"] = pcip_csc["PCIP49_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP50"] == 0, "PCIP50_desc"] = pcip_csc["PCIP50_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP51"] == 0, "PCIP51_desc"] = pcip_csc["PCIP51_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP52"] == 0, "PCIP52_desc"] = pcip_csc["PCIP52_desc"].fillna(0)
# pcip_csc.loc[pcip_csc["PCIP54"] == 0, "PCIP54_desc"] = pcip_csc["PCIP54_desc"].fillna(0)

# pcip_csc[["UNITID","PCIP01_desc"]].groupby("PCIP01_desc").count()
# pcip_csc[["UNITID","PCIP03_desc"]].groupby("PCIP03_desc").count()
# pcip_csc[["UNITID","PCIP04_desc"]].groupby("PCIP04_desc").count()
# pcip_csc[["UNITID","PCIP05_desc"]].groupby("PCIP05_desc").count()
# pcip_csc[["UNITID","PCIP09_desc"]].groupby("PCIP09_desc").count()
# pcip_csc[["UNITID","PCIP10_desc"]].groupby("PCIP10_desc").count()
# pcip_csc[["UNITID","PCIP11_desc"]].groupby("PCIP11_desc").count()
# pcip_csc[["UNITID","PCIP12_desc"]].groupby("PCIP12_desc").count()
# pcip_csc[["UNITID","PCIP13_desc"]].groupby("PCIP13_desc").count()
# pcip_csc[["UNITID","PCIP14_desc"]].groupby("PCIP14_desc").count()
# pcip_csc[["UNITID","PCIP15_desc"]].groupby("PCIP15_desc").count()
# pcip_csc[["UNITID","PCIP16_desc"]].groupby("PCIP16_desc").count()
# pcip_csc[["UNITID","PCIP19_desc"]].groupby("PCIP19_desc").count()
# pcip_csc[["UNITID","PCIP22_desc"]].groupby("PCIP22_desc").count()
# pcip_csc[["UNITID","PCIP23_desc"]].groupby("PCIP23_desc").count()
# pcip_csc[["UNITID","PCIP24_desc"]].groupby("PCIP24_desc").count()
# pcip_csc[["UNITID","PCIP25_desc"]].groupby("PCIP25_desc").count()
# pcip_csc[["UNITID","PCIP26_desc"]].groupby("PCIP26_desc").count()
# pcip_csc[["UNITID","PCIP27_desc"]].groupby("PCIP27_desc").count()
# pcip_csc[["UNITID","PCIP30_desc"]].groupby("PCIP30_desc").count()
# pcip_csc[["UNITID","PCIP31_desc"]].groupby("PCIP31_desc").count()
# pcip_csc[["UNITID","PCIP38_desc"]].groupby("PCIP38_desc").count()
# pcip_csc[["UNITID","PCIP39_desc"]].groupby("PCIP39_desc").count()
# pcip_csc[["UNITID","PCIP40_desc"]].groupby("PCIP40_desc").count()
# pcip_csc[["UNITID","PCIP41_desc"]].groupby("PCIP41_desc").count()
# pcip_csc[["UNITID","PCIP42_desc"]].groupby("PCIP42_desc").count()
# pcip_csc[["UNITID","PCIP43_desc"]].groupby("PCIP43_desc").count()
# pcip_csc[["UNITID","PCIP44_desc"]].groupby("PCIP44_desc").count()
# pcip_csc[["UNITID","PCIP45_desc"]].groupby("PCIP45_desc").count()
# pcip_csc[["UNITID","PCIP46_desc"]].groupby("PCIP46_desc").count()
# pcip_csc[["UNITID","PCIP47_desc"]].groupby("PCIP47_desc").count()
# pcip_csc[["UNITID","PCIP48_desc"]].groupby("PCIP48_desc").count()
# pcip_csc[["UNITID","PCIP49_desc"]].groupby("PCIP49_desc").count()
# pcip_csc[["UNITID","PCIP50_desc"]].groupby("PCIP50_desc").count()
# pcip_csc[["UNITID","PCIP51_desc"]].groupby("PCIP51_desc").count()
# pcip_csc[["UNITID","PCIP52_desc"]].groupby("PCIP52_desc").count()
# pcip_csc[["UNITID","PCIP54_desc"]].groupby("PCIP54_desc").count()

# pcip_csc[["UNITID","PCIP01"]].groupby("PCIP01").count()
# pcip_csc[["UNITID","PCIP03"]].groupby("PCIP03").count()
# pcip_csc[["UNITID","PCIP04"]].groupby("PCIP04").count()
# pcip_csc[["UNITID","PCIP05"]].groupby("PCIP05").count()
# pcip_csc[["UNITID","PCIP09"]].groupby("PCIP09").count()
# pcip_csc[["UNITID","PCIP10"]].groupby("PCIP10").count()
# pcip_csc[["UNITID","PCIP11"]].groupby("PCIP11").count()
# pcip_csc[["UNITID","PCIP12"]].groupby("PCIP12").count()
# pcip_csc[["UNITID","PCIP13"]].groupby("PCIP13").count()
# pcip_csc[["UNITID","PCIP14"]].groupby("PCIP14").count()
# pcip_csc[["UNITID","PCIP15"]].groupby("PCIP15").count()
# pcip_csc[["UNITID","PCIP16"]].groupby("PCIP16").count()
# pcip_csc[["UNITID","PCIP19"]].groupby("PCIP19").count()
# pcip_csc[["UNITID","PCIP22"]].groupby("PCIP22").count()
# pcip_csc[["UNITID","PCIP23"]].groupby("PCIP23").count()
# pcip_csc[["UNITID","PCIP24"]].groupby("PCIP24").count()
# pcip_csc[["UNITID","PCIP25"]].groupby("PCIP25").count()
# pcip_csc[["UNITID","PCIP26"]].groupby("PCIP26").count()
# pcip_csc[["UNITID","PCIP27"]].groupby("PCIP27").count()
# pcip_csc[["UNITID","PCIP30"]].groupby("PCIP30").count()
# pcip_csc[["UNITID","PCIP31"]].groupby("PCIP31").count()
# pcip_csc[["UNITID","PCIP38"]].groupby("PCIP38").count()
# pcip_csc[["UNITID","PCIP39"]].groupby("PCIP39").count()
# pcip_csc[["UNITID","PCIP40"]].groupby("PCIP40").count()
# pcip_csc[["UNITID","PCIP41"]].groupby("PCIP41").count()
# pcip_csc[["UNITID","PCIP42"]].groupby("PCIP42").count()
# pcip_csc[["UNITID","PCIP43"]].groupby("PCIP43").count()
# pcip_csc[["UNITID","PCIP44"]].groupby("PCIP44").count()
# pcip_csc[["UNITID","PCIP45"]].groupby("PCIP45").count()
# pcip_csc[["UNITID","PCIP46"]].groupby("PCIP46").count()
# pcip_csc[["UNITID","PCIP47"]].groupby("PCIP47").count()
# pcip_csc[["UNITID","PCIP48"]].groupby("PCIP48").count()
# pcip_csc[["UNITID","PCIP49"]].groupby("PCIP49").count()
# pcip_csc[["UNITID","PCIP50"]].groupby("PCIP50").count()
# pcip_csc[["UNITID","PCIP51"]].groupby("PCIP51").count()
# pcip_csc[["UNITID","PCIP52"]].groupby("PCIP52").count()
# pcip_csc[["UNITID","PCIP54"]].groupby("PCIP54").count()

p9.ggplot() + p9.geom_line(pcip_csc[["UNITID","PCIP52", "PCIP52_desc"]], p9.aes(x = "PCIP52", y = "PCIP52_desc")) + p9.theme_bw()

In [ ]:
pcip_csc[["PCIP52", "PCIP52_desc"]]

In [ ]:
y["PCIP01_desc", "PCIP03_desc", "PCIP04_desc", "PCIP05_desc", "PCIP09_desc", "PCIP10_desc", "PCIP11_desc", "PCIP12_desc", "PCIP13_desc", "PCIP14_desc", "PCIP15_desc", "PCIP16_desc", "PCIP19_desc", "PCIP22_desc", "PCIP23_desc", "PCIP24_desc", "PCIP25_desc", "PCIP26_desc", "PCIP27_desc", "PCIP30_desc", "PCIP31_desc", "PCIP38_desc", "PCIP39_desc", "PCIP40_desc", "PCIP41_desc", "PCIP42_desc", "PCIP43_desc", "PCIP44_desc", "PCIP45_desc", "PCIP46_desc", "PCIP47_desc", "PCIP48_desc", "PCIP49_desc", "PCIP50_desc", "PCIP51_desc", "PCIP52_desc", "PCIP54_desc"]

In [ ]:
print("cip_csc")
print("INITIAL BEFORE:", b4_cip)
print("INITIAL AFTER: ", af_cip)
print("POST-ENCODING: ", cip_csc.shape)
print(" ")
print("geolocation_csc")
print("INITIAL BEFORE:", b4_geolocation_csc)
print("INITIAL AFTER: ", af_geolocation_csc)
print("POST-ENCODING: ", geolocation_csc.shape)
print(" ")
print("inst_demographic_csc")
print("INITIAL BEFORE:", b4_inst_demographic_csc)
print("INITIAL AFTER: ", af_inst_demographic_csc)
print("POST-ENCODING: ", inst_demographic_csc.shape)
print(" ")
print("stud_demographic_csc")
print("INITIAL BEFORE:", b4_stud_demographic_csc)
print("INITIAL AFTER: ", af_stud_demographic_csc)
print("POST-ENCODING: ", stud_demographic_csc.shape)
print(" ")
print("pcip_csc")
print("INITIAL BEFORE:", b4_pcip_csc)
print("INITIAL AFTER: ", af_pcip_csc)
print("POST-ENCODING: ", pcip_csc.shape)
print(" ")
print("num_csc")
print("INITIAL BEFORE:", b4_num_csc)
print("INITIAL AFTER: ", af_num_csc)
print("POST-ENCODING: ", num_csc.shape)


In [ ]:
cip_csc
geolocation_csc
inst_demographic_csc
stud_demographic_csc
pcip_csc
num_csc

In [ ]:
id_csc

cip_csc
geolocation_csc
inst_demographic_csc
stud_demographic_csc
pcip_csc
num_csc

target_variable.head(3)
inst_demographic_csc.head(3)

In [ ]:
full_df = pd.merge(cip_csc, geolocation_csc.drop(["INSTNM", "ids"], axis = 1), how = 'left',
                   left_on = "UNITID", right_on = "UNITID",
                   suffixes=("_cip", "_geo"))

full_df = pd.merge(full_df, inst_demographic_csc.drop(["INSTNM", "ids"], axis = 1), how = 'left',
                   left_on = "UNITID", right_on = "UNITID",
                   suffixes=("_cip", "_geo"))

full_df = pd.merge(full_df, stud_demographic_csc.drop(["INSTNM", "ids"], axis = 1), how = 'left',
                   left_on = "UNITID", right_on = "UNITID",
                   suffixes=("_cip", "_geo"))

full_df = pd.merge(full_df, pcip_csc.drop(["INSTNM", "ids"], axis = 1), how = 'left',
                   left_on = "UNITID", right_on = "UNITID",
                   suffixes=("_cip", "_geo"))

full_df = pd.merge(full_df, num_csc.drop(["INSTNM", "ids"], axis = 1), how = 'left',
                   left_on = "UNITID", right_on = "UNITID",
                   suffixes=("_cip", "_geo"))

In [ ]:
full_df #.head(5)

missing = pd.DataFrame(full_df.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] != 0].count())
missing.to_csv("miss_fulldf.csv")

In [ ]:
# feather.write_feather(full_df, full_df_path)

# cip_csc

In [ ]:
enc_assoc = OneHotEncoder(handle_unknown="ignore")

enc_assoc.fit(np.array(cip_csc["CIP01ASSOC"]).reshape(-1, 1))

labels = np.array(cip_csc["CIP01ASSOC"])

encoded = enc_assoc.transform(labels.reshape(-1,1)).toarray()

print("      Original shape:", cip_csc.shape)
print("          Categories:", enc_assoc.categories_)
print("       Encoded shape:", encoded.shape)

cip_csc["CIP01ASSOC_encoded0"] = encoded[:, 0]
cip_csc["CIP01ASSOC_encoded1"] = encoded[:, 1]
cip_csc["CIP01ASSOC_encoded2"] = encoded[:, 2]

print("Shape after encoding:", cip_csc.shape)

lst = ["UNITID", "INSTNM", "CIP01ASSOC", "CIP01ASSOC_encoded0", "CIP01ASSOC_encoded1", "CIP01ASSOC_encoded2"]

pd.concat([cip_csc.loc[:, lst].query("CIP01ASSOC == 0").head(1),
           cip_csc.loc[:, lst].query("CIP01ASSOC == 1").head(1),
           cip_csc.loc[:, lst].query("CIP01ASSOC == 2").head(1)])

In [ ]:
enc_bach = OneHotEncoder(handle_unknown="ignore")

enc_bach.fit(np.array(cip_csc["CIP01BACHL"]).reshape(-1, 1))

labels = np.array(cip_csc["CIP01BACHL"])

encoded = enc_bach.transform(labels.reshape(-1,1)).toarray()

print("      Original shape:", cip_csc.shape)
print("          Categories:", enc_bach.categories_)
print("       Encoded shape:", encoded.shape)

cip_csc["CIP01BACHL_encoded0"] = encoded[:, 0]
cip_csc["CIP01BACHL_encoded1"] = encoded[:, 1]
cip_csc["CIP01BACHL_encoded2"] = encoded[:, 2]

print("Shape after encoding:", cip_csc.shape)

lst = ["UNITID", "INSTNM", "CIP01BACHL", "CIP01BACHL_encoded0", "CIP01BACHL_encoded1", "CIP01BACHL_encoded2"]

pd.concat([cip_csc.loc[:, lst].query("CIP01BACHL == 0").head(1),
           cip_csc.loc[:, lst].query("CIP01BACHL == 1").head(1),
           cip_csc.loc[:, lst].query("CIP01BACHL == 2").head(1)])

In [ ]:
enc_CERT1 = OneHotEncoder(handle_unknown="ignore")

enc_CERT1.fit(np.array(cip_csc["CIP01CERT1"]).reshape(-1, 1))

labels = np.array(cip_csc["CIP01CERT1"])

encoded = enc_CERT1.transform(labels.reshape(-1,1)).toarray()

print("      Original shape:", cip_csc.shape)
print("          Categories:", enc_CERT1.categories_)
print("       Encoded shape:", encoded.shape)

cip_csc["CIP01CERT1_encoded0"] = encoded[:, 0]
cip_csc["CIP01CERT1_encoded1"] = encoded[:, 1]
cip_csc["CIP01CERT1_encoded2"] = encoded[:, 2]

print("Shape after encoding:", cip_csc.shape)

lst = ["UNITID", "INSTNM", "CIP01CERT1", "CIP01CERT1_encoded0", "CIP01CERT1_encoded1", "CIP01CERT1_encoded2"]

pd.concat([cip_csc.loc[:, lst].query("CIP01CERT1 == 0").head(1),
           cip_csc.loc[:, lst].query("CIP01CERT1 == 1").head(1),
           cip_csc.loc[:, lst].query("CIP01CERT1 == 2").head(1)])

In [ ]:
enc_CERT2 = OneHotEncoder(handle_unknown="ignore")

enc_CERT2.fit(np.array(cip_csc["CIP01CERT2"]).reshape(-1, 1))

labels = np.array(cip_csc["CIP01CERT2"])

encoded = enc_CERT2.transform(labels.reshape(-1,1)).toarray()

print("      Original shape:", cip_csc.shape)
print("          Categories:", enc_CERT2.categories_)
print("       Encoded shape:", encoded.shape)

cip_csc["CIP01CERT2_encoded0"] = encoded[:, 0]
cip_csc["CIP01CERT2_encoded1"] = encoded[:, 1]
cip_csc["CIP01CERT2_encoded2"] = encoded[:, 2]

print("Shape after encoding:", cip_csc.shape)

lst = ["UNITID", "INSTNM", "CIP01CERT2", "CIP01CERT2_encoded0", "CIP01CERT2_encoded1", "CIP01CERT2_encoded2"]

pd.concat([cip_csc.loc[:, lst].query("CIP01CERT2 == 0").head(1),
           cip_csc.loc[:, lst].query("CIP01CERT2 == 1").head(1),
           cip_csc.loc[:, lst].query("CIP01CERT2 == 2").head(1)])

In [ ]:
num_category = ["CIP01CERT4", "CIP01CERT2", "CIP01CERT1", "CIP01BACHL", "CIP01ASSOC"]

name_list = []

for x in num_category:
    name_list.append([(x + "_encoded{}").format(i) for i, cat in enumerate(np.array(enc_CERT2.categories_).transpose(), 0)])
name_list

In [ ]:
num_category = ["CIP01CERT4"]
# name_list = []
# for x in num_category:
    # name_list.append([(x + "_encoded{}").format(i) for i in range(3)])

name_list = np.array([(x + "_encoded{}").format(i) for i in range(3)])
name_list[1]

In [ ]:
enc_CERT4 = OneHotEncoder(handle_unknown="ignore")

enc_CERT4.fit(np.array(cip_csc["CIP01CERT4"]).reshape(-1, 1))

labels = np.array(cip_csc["CIP01CERT4"])

encoded = enc_CERT4.transform(labels.reshape(-1,1)).toarray()

print("      Original shape:", cip_csc.shape)
print("          Categories:", enc_CERT4.categories_)
print("       Encoded shape:", encoded.shape)

cip_csc["CIP01CERT4_encoded0"] = encoded[:, 0]
cip_csc["CIP01CERT4_encoded1"] = encoded[:, 1]
# cip_csc["CIP01CERT4_encoded2"] = encoded[:, 2]

print("Shape after encoding:", cip_csc.shape)

lst = ["UNITID", "INSTNM", "CIP01CERT4", "CIP01CERT4_encoded0", "CIP01CERT4_encoded1"] #, "CIP01CERT4_encoded2"]

pd.concat([cip_csc.loc[:, lst].query("CIP01CERT4 == 0").head(1),
           cip_csc.loc[:, lst].query("CIP01CERT4 == 1").head(1)])


# geolocation_csc

In [ ]:
geolocation_csc.head(3)

In [ ]:
missing = pd.DataFrame(geolocation_csc.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] != 0].count())
missing

In [ ]:
encoder = TargetEncoder()
encoder.fit(geolocation_csc["CITY"], id_csc["PELLCAT"])
geolocation_csc["CITY Encoded"] = encoder.transform(geolocation_csc["CITY"])
geolocation_csc.loc[:,["STABBR", "CITY", "CITY Encoded"]]


encoder = TargetEncoder()
encoder.fit(geolocation_csc["INSTNM"], id_csc["PELLCAT"])
geolocation_csc["INSTNM Encoded"] = encoder.transform(geolocation_csc["INSTNM"])
geolocation_csc.loc[:,["STABBR", "INSTNM", "INSTNM Encoded"]]

geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["ST_FIPS"].astype(int), prefix = "FIPS")],
                             axis = 1)

geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["REGION"].astype(int), prefix = "REGION")],
                             axis = 1)

geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["LOCALE"].astype(int), prefix = "LOC")],
                             axis = 1)
####
geolocation_csc.loc[geolocation_csc["CCBASIC"] == 99, "CCBASIC"] = 99
geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["CCBASIC"].astype(int), prefix = "CCB")],
                             axis = 1)
####
geolocation_csc.loc[geolocation_csc["CCSIZSET"] == 99, "CCSIZSET"] = 99
geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["CCSIZSET"].astype(int), prefix = "CCSS")],
                             axis = 1)
####
geolocation_csc.loc[geolocation_csc["CCUGPROF"] == 99, "CCUGPROF"] = 99
geolocation_csc = pd.concat([geolocation_csc,
                             pd.get_dummies(geolocation_csc["CCUGPROF"].astype(int), prefix = "CCPROF")],
                             axis = 1)
####
geolocation_csc["ZIP5"] = geolocation_csc["ZIP"].str.slice(0, 5)
geolocation_csc["ZIP4"] = geolocation_csc["ZIP"].str.slice(0, 4)
geolocation_csc["ZIP3"] = geolocation_csc["ZIP"].str.slice(0, 3)
geolocation_csc["ZIP2"] = geolocation_csc["ZIP"].str.slice(0, 2)

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP2"], id_csc["PELLCAT"])
geolocation_csc["ZIP2 Encoded"] = encoder.transform(geolocation_csc["ZIP2"])
geolocation_csc.loc[:,["STABBR", "ZIP2", "ZIP2 Encoded"]]

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP3"], id_csc["PELLCAT"])
geolocation_csc["ZIP3 Encoded"] = encoder.transform(geolocation_csc["ZIP3"])
geolocation_csc.loc[:,["STABBR", "ZIP3", "ZIP3 Encoded"]]

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP4"], id_csc["PELLCAT"])
geolocation_csc["ZIP4 Encoded"] = encoder.transform(geolocation_csc["ZIP4"])
geolocation_csc.loc[:,["STABBR", "ZIP4", "ZIP4 Encoded"]]

encoder = TargetEncoder()
encoder.fit(geolocation_csc["ZIP5"], id_csc["PELLCAT"])
geolocation_csc["ZIP5 Encoded"] = encoder.transform(geolocation_csc["ZIP5"])
geolocation_csc.loc[:,["STABBR", "ZIP5", "ZIP5 Encoded"]]

In [ ]:
encoder = TargetEncoder()
encoder.fit(geolocation_csc["INSTNM"], id_csc["PELLCAT"])
geolocation_csc["INSTNM Encoded"] = encoder.transform(geolocation_csc["INSTNM"])
geolocation_csc.loc[:,["STABBR", "INSTNM", "INSTNM Encoded"]]

In [ ]:
var_names = geolocation_csc.drop(columns = ["ids", "UNITID", "INSTNM", "LATITUDE", "LONGITUDE", "CITY", "STABBR"], axis = 1)

print("Original shape:", geolocation_csc.shape) # Original shape: (5879, 193)
print("Number of variables that will be encoded:", var_names.shape) # (5879, 190)

nber = 0
for x in list(var_names):
    enc = OneHotEncoder(handle_unknown="ignore")
    enc.fit(np.array(geolocation_csc[x]).reshape(-1, 1))
    num_category = len(np.array(enc.categories_).transpose())
    nber = nber + num_category

print("Number of variables to expect:", geolocation_csc.shape[1] + nber)

for x in list(var_names):
    enc = OneHotEncoder(handle_unknown="ignore")

    enc.fit(np.array(geolocation_csc[x]).reshape(-1, 1))
    labels = np.array(geolocation_csc[x])
    encoded = enc.transform(labels.reshape(-1,1)).toarray()

    num_category = len(np.array(enc.categories_).transpose())
    var_name = np.array([(x + "_encoded{}").format(i) for i in range(num_category)])

    for i in range(len(var_name)):
        geolocation_csc[var_name[i]] = encoded[:, i]
geolocation_csc.shape

# inst_demographic_csc

In [ ]:
missing = pd.DataFrame(inst_demographic_csc.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] != 0].count())
# missing[missing[0] > 0]
missing

In [ ]:


print(len(inst_demographic_csc.groupby("T4APPROVALDATE")["T4APPROVALDATE"].count()))
inst_demographic_csc.groupby("T4APPROVALDATE")["T4APPROVALDATE"].count()

# inst_demographic_csc["T4APPROVALDATE"] = inst_demographic_csc["T4APPROVALDATE"].str.split("/")
# inst_demographic_csc["T4APPROVALDATE"]
# inst_demographic_csc["T4APPROVALDATE"].str.slice(0,1)
# inst_demographic_csc["T4APPROVALDATE"].str.slice(1,2)




print(len(inst_demographic_csc.groupby("Season")["Season"].count()))
inst_demographic_csc.groupby("Season")["Season"].count()

# inst_demographic_csc = pd.concat([inst_demographic_csc,
#                              pd.get_dummies(inst_demographic_csc["MAIN"].astype(int), prefix = "MAIN")],
#                              axis = 1)

# encoder = TargetEncoder()
# encoder.fit(inst_demographic_csc["ACCREDAGENCY"], id_csc["PELLCAT"])
# inst_demographic_csc["ACCREDAGENCY Encoded"] = encoder.transform(inst_demographic_csc["ACCREDAGENCY"])
# inst_demographic_csc.loc[:,["INSTNM", "ACCREDAGENCY", "ACCREDAGENCY Encoded"]]

In [ ]:
inst_demographic_csc = pd.concat((inst_demographic_csc,
                                  pd.DataFrame(np.array(list(inst_demographic_csc["T4APPROVALDATE"].str.split("/"))),
                                               columns = ["T4_month", "T4_day", "T4_year"])),
                                  axis = 1)

inst_demographic_csc.loc[:, ["INSTNM", "T4APPROVALDATE", "T4_month", "T4_day", "T4_year"]]
inst_demographic_csc["Season"] = inst_demographic_csc["T4_month"].map(month_dict)

In [ ]:
inst_demographic_csc = pd.concat((inst_demographic_csc,
                                  pd.DataFrame(np.array(list(inst_demographic_csc["T4APPROVALDATE"].str.split("/"))),
                                               columns = ["T4_month", "T4_day", "T4_year"])),
                                  axis = 1)

inst_demographic_csc.loc[:, ["INSTNM", "T4APPROVALDATE", "T4_month", "T4_day", "T4_year"]]
inst_demographic_csc["Season"] = inst_demographic_csc["T4_month"].map(month_dict)                                 

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["MAIN"].astype(int),
                                  prefix = "MAIN")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["HCM2"].astype(int),
                                  prefix = "HCM2")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["OPEFLAG"].astype(int),
                                  prefix = "OPEFLAG")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["PREDDEG"].astype(int),
                                  prefix = "PREDDEG")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["HIGHDEG"].astype(int),
                                  prefix = "HIGHDEG")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["ICLEVEL"].astype(int),
                                  prefix = "ICLEVEL")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["CONTROL"],
                                  prefix = "CONTROL")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["CURROPER"].astype(int),
                                  prefix = "CURROPER")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["OPENADMP"].astype(int),
                                  prefix = "OPENADMP")],
                                  axis = 1)

inst_demographic_csc = pd.concat([inst_demographic_csc,
                                  pd.get_dummies(inst_demographic_csc["ACCREDCODE"],
                                  prefix = "ACCRED")],
                                  axis = 1)

encoder = TargetEncoder()
encoder.fit(inst_demographic_csc["ACCREDAGENCY"], id_csc["PELLCAT"])
inst_demographic_csc["ACCREDAGENCY Encoded"] = encoder.transform(inst_demographic_csc["ACCREDAGENCY"])

encoder = TargetEncoder()
encoder.fit(inst_demographic_csc["Season"], id_csc["PELLCAT"])
inst_demographic_csc["Season Encoded"] = encoder.transform(inst_demographic_csc["Season"])

# stud_demographic_csc

In [ ]:
missing = pd.DataFrame(stud_demographic_csc.isna().sum())
missing.reset_index(inplace=True)
print(missing[missing[0] != 0].count())
missing[missing[0] > 0]
# missing

In [ ]:
stud_demographic_csc["HSI"].fillna(0, inplace=True)
stud_demographic_csc["HBCU"].fillna(0, inplace=True)
stud_demographic_csc["ANNHI"].fillna(0, inplace=True)
stud_demographic_csc["PBI"].fillna(0, inplace=True)
stud_demographic_csc["TRIBAL"].fillna(0, inplace=True)
stud_demographic_csc["NANTI"].fillna(0, inplace=True)
stud_demographic_csc["AANAPII"].fillna(0, inplace=True)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["HSI"],
                                  prefix = "HSI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["HBCU"],
                                  prefix = "HBCU")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["ANNHI"],
                                  prefix = "ANNHI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["PBI"],
                                  prefix = "PBI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["TRIBAL"],
                                  prefix = "TRIBAL")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["NANTI"],
                                  prefix = "NANTI")],
                                  axis = 1)

stud_demographic_csc = pd.concat([stud_demographic_csc,
                                  pd.get_dummies(stud_demographic_csc["AANAPII"],
                                  prefix = "AANAPII")],
                                  axis = 1)

In [ ]:
stud_demographic_csc.head(5)

# pcip_csc

In [ ]:
pcip_csc

In [ ]:
b = [0, 0.125, 0.325, 0.525, 0.725, 0.925]
n, bns, patches = plt.hist(pcip_csc['PCIP51'], b)
bns = [0.125, 0.325, 0.525, 0.725, 0.925]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.125, 0.325, 0.525, 0.725, 0.925]
n, bns, patches = plt.hist(pcip_csc['PCIP52'], b)
bns = [0.125, 0.325, 0.525, 0.725, 0.925]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.125, 0.325, 0.525, 0.725, 0.925]
n, bns, patches = plt.hist(pcip_csc['PCIP01'], b)
bns = [0.125, 0.325, 0.525, 0.725, 0.925]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.125, 0.325, 0.525, 0.725, 0.925]
n, bns, patches = plt.hist(pcip_csc['PCIP54'], b)
bns = [0.125, 0.325, 0.525, 0.725, 0.925]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.03, 0.05, 0.07, 0.09, 0.11]
n, bns, patches = plt.hist(pcip_csc['PCIP11'], b)
bns = [0.03, 0.05, 0.07, 0.09, 0.11]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12]
n, bns, patches = plt.hist(pcip_csc['PCIP09'], b)
bns = [0.02, 0.04, 0.06, 0.08, 0.1, 0.12]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.007, 0.0175, 0.028, 0.0385, 0.049]
n, bns, patches = plt.hist(pcip_csc['PCIP03'], b)
bns = [0.007, 0.0175, 0.028, 0.0385, 0.049]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0.0000000000001, 0.005, 0.0125, 0.02, 0.0275, 0.035]
n, bns, patches = plt.hist(pcip_csc['PCIP04'], b)
bns = [0.005, 0.0125, 0.02, 0.0275, 0.035]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.002, 0.0035, 0.005, 0.0065, 0.008]
n, bns, patches = plt.hist(pcip_csc['PCIP05'], b)
bns = [0.002, 0.0035, 0.005, 0.0065, 0.008]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004]
n, bns, patches = plt.hist(pcip_csc['PCIP10'], b)
bns = [0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475]
n, bns, patches = plt.hist(pcip_csc['PCIP13'], b)
bns = [0.0010, 0.001625, 0.002225, 0.00285, 0.003475]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001]
n, bns, patches = plt.hist(pcip_csc['PCIP14'], b)
bns = [0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001]
n, bns, patches = plt.hist(pcip_csc['PCIP15'], b)
bns = [0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
##########################################################################
b = [0, 0.001, 0.004, 0.007, 0.01]
n, bns, patches = plt.hist(pcip_csc['PCIP16'], b)
bns = [0.001, 0.004, 0.007, 0.01]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.005, 0.035, 0.065, 0.095]
n, bns, patches = plt.hist(pcip_csc['PCIP19'], b)
bns = [0.005, 0.035, 0.065, 0.095]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.01, 0.035, 0.065, 0.095]
n, bns, patches = plt.hist(pcip_csc['PCIP22'], b)
bns = [0.01, 0.035, 0.065, 0.095]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.001, 0.012, 0.023, 0.034]
n, bns, patches = plt.hist(pcip_csc['PCIP23'], b)
bns = [0.001, 0.012, 0.023, 0.034]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481]
n, bns, patches = plt.hist(pcip_csc['PCIP24'], b)
bns = [0.001, 0.0121, 0.0241, 0.0361, 0.0481]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135]
n, bns, patches = plt.hist(pcip_csc['PCIP25'], b)
bns = [0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw()
#########################################################################
b = [0, 0.01, 0.04, 0.07, 0.1, 0.13]
n, bns, patches = plt.hist(pcip_csc['PCIP26'], b)
bns = [0.01, 0.04, 0.07, 0.1, 0.13]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0025, 0.01, 0.0175, 0.025, 0.0325]
n, bns, patches = plt.hist(pcip_csc['PCIP27'], b)
bns = [0.0025, 0.01, 0.0175, 0.025, 0.0325]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046]
n, bns, patches = plt.hist(pcip_csc['PCIP30'], b)
bns = [0.001, 0.01, 0.019, 0.028, 0.037, 0.046]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046]
n, bns, patches = plt.hist(pcip_csc['PCIP31'], b)
bns = [0.001, 0.01, 0.019, 0.028, 0.037, 0.046]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036]
n, bns, patches = plt.hist(pcip_csc['PCIP38'], b)
bns = [0.001, 0.008, 0.015, 0.022, 0.029, 0.036]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275]
n, bns, patches = plt.hist(pcip_csc['PCIP39'], b)
bns = [0.00075, 0.00625, 0.01175, 0.01725, 0.02275]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.001, 0.006, 0.011, 0.016, 0.021]
n, bns, patches = plt.hist(pcip_csc['PCIP40'], b)
bns = [0.001, 0.006, 0.011, 0.016, 0.021]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0005, 0.003, 0.0055, 0.008, 0.0105]
n, bns, patches = plt.hist(pcip_csc['PCIP41'], b)
bns = [0.0005, 0.003, 0.0055, 0.008, 0.0105]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11]
n, bns, patches = plt.hist(pcip_csc['PCIP42'], b)
bns = [0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.01, 0.03, 0.05, 0.07, 0.09]
n, bns, patches = plt.hist(pcip_csc['PCIP43'], b)
bns = [0.01, 0.03, 0.05, 0.07, 0.09]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.005, 0.02, 0.035, 0.05, 0.065]
n, bns, patches = plt.hist(pcip_csc['PCIP45'], b)
bns = [0.005, 0.02, 0.035, 0.05, 0.065]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.005, 0.02, 0.035, 0.05, 0.065]
n, bns, patches = plt.hist(pcip_csc['PCIP44'], b)
bns = [0.005, 0.02, 0.035, 0.05, 0.065]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0103, 0.037796, 0.125, 1]
n, bns, patches = plt.hist(pcip_csc['PCIP46'], b)
bns = [0.0103, 0.037796, 0.125, 1]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615]
n, bns, patches = plt.hist(pcip_csc['PCIP47'], b)
bns = [0.0005, 0.0165, 0.0315, 0.0465, 0.0615]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005]
n, bns, patches = plt.hist(pcip_csc['PCIP48'], b)
bns = [0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005]
n, bns, patches = plt.hist(pcip_csc['PCIP49'], b)
bns = [0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw() 
#########################################################################
b = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805]
n, bns, patches = plt.hist(pcip_csc['PCIP50'], b)
bns = [0.0005, 0.0205, 0.0405, 0.0605, 0.0805]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw()
#########################################################################
b = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805]
n, bns, patches = plt.hist(pcip_csc['PCIP12'], b)
bns = [0.0005, 0.0205, 0.0405, 0.0605, 0.0805]
df = pd.DataFrame(bns, n)
p9.ggplot(df, p9.aes(x = bns, y = n)) + p9.geom_line() + p9.theme_bw()

In [ ]:
# pcip_csc["PCIP01_desc"]=pd.cut(pcip_csc["PCIP01"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"])
# pcip_csc["PCIP03_desc"]=pd.cut(pcip_csc["PCIP03"],bins = [0, 0.007, 0.0175, 0.028, 0.0385, 0.049, 1],labels = ["0%", "0.7%","1.75%","2.8%","3.85%","4.9%"])
# pcip_csc["PCIP04_desc"]=pd.cut(pcip_csc["PCIP04"],bins = [0, 0.005, 0.0125, 0.02, 0.0275, 0.035, 1],labels = ["0%", "0.5%","1.25%","2%","2.75%","3.5%"])
# pcip_csc["PCIP05_desc"]=pd.cut(pcip_csc["PCIP05"],bins = [0, 0.002, 0.0035, 0.005, 0.0065, 0.008, 1],labels = ["0%", "0.2%","0.35%","0.5%","0.65%","0.8%"])
# pcip_csc["PCIP09_desc"]=pd.cut(pcip_csc["PCIP09"],bins = [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 1],labels = ["0%", "2%","4%","6%","8%","10%","12%"])
# pcip_csc["PCIP10_desc"]=pd.cut(pcip_csc["PCIP10"],bins = [0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004, 1],labels = ["0%", "0.11%","0.18%","0.24%","0.29%","0.35%","0.4%"])
# pcip_csc["PCIP11_desc"]=pd.cut(pcip_csc["PCIP11"],bins = [0, 0.03, 0.05, 0.07, 0.09, 0.11, 1],labels = ["0%", "3%","5%","7%","9%","11%"])
# pcip_csc["PCIP12_desc"]=pd.cut(pcip_csc["PCIP12"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1],labels = ["0%", "0.05%","2.05%","4.05%","6.05%","8.05%"])
# pcip_csc["PCIP13_desc"]=pd.cut(pcip_csc["PCIP13"],bins = [0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475, 1],labels = ["0%", "0.1%","0.16%","0.22%","0.29%","0.35%"])
# pcip_csc["PCIP14_desc"]=pd.cut(pcip_csc["PCIP14"],bins = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1],labels = ["0%", "3.8%","7.6%","11.4%","15.2%","19%", "100%"])
# pcip_csc["PCIP15_desc"]=pd.cut(pcip_csc["PCIP15"],bins = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1],labels = ["0%", "3.8%","7.6%","11.4%","15.2%","19%", "100%"])
# pcip_csc["PCIP16_desc"]=pd.cut(pcip_csc["PCIP16"],bins = [0, 0.001, 0.004, 0.007, 0.01, 1],labels = ["0%", "0.1%","0.4%","0.7%","1%"])
# pcip_csc["PCIP19_desc"]=pd.cut(pcip_csc["PCIP19"],bins = [0, 0.005, 0.035, 0.065, 0.095, 1],labels = ["0%", "0.5%","3.5%","6.5%","9.5%"])
# pcip_csc["PCIP22_desc"]=pd.cut(pcip_csc["PCIP22"],bins = [0, 0.01, 0.035, 0.065, 0.095, 1],labels = ["0%", "1%","3.5%","6.5%","9.5%"])
# pcip_csc["PCIP23_desc"]=pd.cut(pcip_csc["PCIP23"],bins = [0, 0.001, 0.012, 0.023, 0.034, 1],labels = ["0%", "0.1%","1.2%","2.3%","3.4%"])
# pcip_csc["PCIP24_desc"]=pd.cut(pcip_csc["PCIP24"],bins = [0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481, 1],labels = ["0%", "0.1%","1.21%","2.41%","3.61%","4.81%"])
# pcip_csc["PCIP25_desc"]=pd.cut(pcip_csc["PCIP25"],bins = [0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135, 1],labels = ["0%", "0.1%","0.35%","0.6%","0.85%","1.1%","1.35%"])
# pcip_csc["PCIP26_desc"]=pd.cut(pcip_csc["PCIP26"],bins = [0, 0.01, 0.04, 0.07, 0.1, 0.13, 1],labels = ["0%", "1%","4%","7%","10%","13%"])
# pcip_csc["PCIP27_desc"]=pd.cut(pcip_csc["PCIP27"],bins = [0, 0.0025, 0.01, 0.0175, 0.025, 0.0325, 1],labels = ["0%", "0.25%","1%","1.75%","2.5%","3.25%"])
# pcip_csc["PCIP30_desc"]=pd.cut(pcip_csc["PCIP30"],bins = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1],labels = ["0%", "0.1%","1%","1.9%","2.8%","3.7%","4.6%"])
# pcip_csc["PCIP31_desc"]=pd.cut(pcip_csc["PCIP31"],bins = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1],labels = ["0%", "0.1%","1%","1.9%","2.8%","3.7%","4.6%"])
# pcip_csc["PCIP38_desc"]=pd.cut(pcip_csc["PCIP38"],bins = [0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036, 1],labels = ["0%", "0.1%","0.8%","1.5%","2.2%","2.9%","3.6%"])
# pcip_csc["PCIP39_desc"]=pd.cut(pcip_csc["PCIP39"],bins = [0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275, 1],labels = ["0%", "0.08%","0.63%","1.18%","1.73%","2.28%"])
# pcip_csc["PCIP40_desc"]=pd.cut(pcip_csc["PCIP40"],bins = [0, 0.001, 0.006, 0.011, 0.016, 0.021, 1],labels = ["0%", "0.1%","0.6%","1.1%","1.6%","2.1%"])
# pcip_csc["PCIP41_desc"]=pd.cut(pcip_csc["PCIP41"],bins = [0, 0.0005, 0.003, 0.0055, 0.008, 0.0105, 1],labels = ["0%", "0.05%","0.3%","0.55%","0.8%","1.05%"])
# pcip_csc["PCIP42_desc"]=pd.cut(pcip_csc["PCIP42"],bins = [0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11, 1],labels = ["0%", "3.5%","4.75%","6%","7.25%","8.5%","9.75%","11%"])
# pcip_csc["PCIP43_desc"]=pd.cut(pcip_csc["PCIP43"],bins = [0, 0.01, 0.03, 0.05, 0.07, 0.09, 1],labels = ["0%", "1%","3%","5%","7%","9%"])
# pcip_csc["PCIP44_desc"]=pd.cut(pcip_csc["PCIP44"],bins = [0, 0.005, 0.02, 0.035, 0.05, 0.065, 1],labels = ["0%", "0.5%","2%","3.5%","5%","6.5%"])
# pcip_csc["PCIP45_desc"]=pd.cut(pcip_csc["PCIP45"],bins = [0, 0.005, 0.02, 0.035, 0.05, 0.065, 1],labels = ["0%", "0.5%","2%","3.5%","5%","6.5%"])
# pcip_csc["PCIP46_desc"]=pd.cut(pcip_csc["PCIP46"],bins = [0, 0.0103, 0.037796, 0.125, 1],labels = ["0%", "1.03%","3.78%","12.5%"])
# pcip_csc["PCIP47_desc"]=pd.cut(pcip_csc["PCIP47"],bins = [0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615, 1],labels = ["0%", "0.05%","1.65%","3.15%","4.65%","6.15%"])
# pcip_csc["PCIP48_desc"]=pd.cut(pcip_csc["PCIP48"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1],labels = ["0%", "0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"])
# pcip_csc["PCIP49_desc"]=pd.cut(pcip_csc["PCIP49"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1],labels = ["0%", "0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"])
# pcip_csc["PCIP50_desc"]=pd.cut(pcip_csc["PCIP50"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1],labels = ["0%", "0.05%","2.05%","4.05%","6.05%","8.05%"])
# pcip_csc["PCIP51_desc"]=pd.cut(pcip_csc["PCIP51"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"])
# pcip_csc["PCIP52_desc"]=pd.cut(pcip_csc["PCIP52"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"])
# pcip_csc["PCIP54_desc"]=pd.cut(pcip_csc["PCIP54"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"])

# pcip_csc.loc[pcip_csc["PCIP01"] == 0, "PCIP01_desc"] = pcip_csc["PCIP01_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP03"] == 0, "PCIP03_desc"] = pcip_csc["PCIP03_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP04"] == 0, "PCIP04_desc"] = pcip_csc["PCIP04_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP05"] == 0, "PCIP05_desc"] = pcip_csc["PCIP05_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP09"] == 0, "PCIP09_desc"] = pcip_csc["PCIP09_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP10"] == 0, "PCIP10_desc"] = pcip_csc["PCIP10_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP11"] == 0, "PCIP11_desc"] = pcip_csc["PCIP11_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP12"] == 0, "PCIP12_desc"] = pcip_csc["PCIP12_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP13"] == 0, "PCIP13_desc"] = pcip_csc["PCIP13_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP14"] == 0, "PCIP14_desc"] = pcip_csc["PCIP14_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP15"] == 0, "PCIP15_desc"] = pcip_csc["PCIP15_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP16"] == 0, "PCIP16_desc"] = pcip_csc["PCIP16_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP19"] == 0, "PCIP19_desc"] = pcip_csc["PCIP19_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP22"] == 0, "PCIP22_desc"] = pcip_csc["PCIP22_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP23"] == 0, "PCIP23_desc"] = pcip_csc["PCIP23_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP24"] == 0, "PCIP24_desc"] = pcip_csc["PCIP24_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP25"] == 0, "PCIP25_desc"] = pcip_csc["PCIP25_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP26"] == 0, "PCIP26_desc"] = pcip_csc["PCIP26_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP27"] == 0, "PCIP27_desc"] = pcip_csc["PCIP27_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP30"] == 0, "PCIP30_desc"] = pcip_csc["PCIP30_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP31"] == 0, "PCIP31_desc"] = pcip_csc["PCIP31_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP38"] == 0, "PCIP38_desc"] = pcip_csc["PCIP38_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP39"] == 0, "PCIP39_desc"] = pcip_csc["PCIP39_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP40"] == 0, "PCIP40_desc"] = pcip_csc["PCIP40_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP41"] == 0, "PCIP41_desc"] = pcip_csc["PCIP41_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP42"] == 0, "PCIP42_desc"] = pcip_csc["PCIP42_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP43"] == 0, "PCIP43_desc"] = pcip_csc["PCIP43_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP44"] == 0, "PCIP44_desc"] = pcip_csc["PCIP44_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP45"] == 0, "PCIP45_desc"] = pcip_csc["PCIP45_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP46"] == 0, "PCIP46_desc"] = pcip_csc["PCIP46_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP47"] == 0, "PCIP47_desc"] = pcip_csc["PCIP47_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP48"] == 0, "PCIP48_desc"] = pcip_csc["PCIP48_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP49"] == 0, "PCIP49_desc"] = pcip_csc["PCIP49_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP50"] == 0, "PCIP50_desc"] = pcip_csc["PCIP50_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP51"] == 0, "PCIP51_desc"] = pcip_csc["PCIP51_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP52"] == 0, "PCIP52_desc"] = pcip_csc["PCIP52_desc"].fillna("0%")
# pcip_csc.loc[pcip_csc["PCIP54"] == 0, "PCIP54_desc"] = pcip_csc["PCIP54_desc"].fillna("0%")

pcip_csc[["UNITID","PCIP01_desc"]].groupby("PCIP01_desc").count()
# pcip_csc[["UNITID","PCIP03_desc"]].groupby("PCIP03_desc").count()
# pcip_csc[["UNITID","PCIP04_desc"]].groupby("PCIP04_desc").count()
# pcip_csc[["UNITID","PCIP05_desc"]].groupby("PCIP05_desc").count()
# pcip_csc[["UNITID","PCIP09_desc"]].groupby("PCIP09_desc").count()
# pcip_csc[["UNITID","PCIP10_desc"]].groupby("PCIP10_desc").count()
# pcip_csc[["UNITID","PCIP11_desc"]].groupby("PCIP11_desc").count()
# pcip_csc[["UNITID","PCIP12_desc"]].groupby("PCIP12_desc").count()
# pcip_csc[["UNITID","PCIP13_desc"]].groupby("PCIP13_desc").count()
# pcip_csc[["UNITID","PCIP14_desc"]].groupby("PCIP14_desc").count()
# pcip_csc[["UNITID","PCIP15_desc"]].groupby("PCIP15_desc").count()
# pcip_csc[["UNITID","PCIP16_desc"]].groupby("PCIP16_desc").count()
# pcip_csc[["UNITID","PCIP19_desc"]].groupby("PCIP19_desc").count()
# pcip_csc[["UNITID","PCIP22_desc"]].groupby("PCIP22_desc").count()
# pcip_csc[["UNITID","PCIP23_desc"]].groupby("PCIP23_desc").count()
# pcip_csc[["UNITID","PCIP24_desc"]].groupby("PCIP24_desc").count()
# pcip_csc[["UNITID","PCIP25_desc"]].groupby("PCIP25_desc").count()
# pcip_csc[["UNITID","PCIP26_desc"]].groupby("PCIP26_desc").count()
# pcip_csc[["UNITID","PCIP27_desc"]].groupby("PCIP27_desc").count()
# pcip_csc[["UNITID","PCIP30_desc"]].groupby("PCIP30_desc").count()
# pcip_csc[["UNITID","PCIP31_desc"]].groupby("PCIP31_desc").count()
# pcip_csc[["UNITID","PCIP38_desc"]].groupby("PCIP38_desc").count()
# pcip_csc[["UNITID","PCIP39_desc"]].groupby("PCIP39_desc").count()
# pcip_csc[["UNITID","PCIP40_desc"]].groupby("PCIP40_desc").count()
# pcip_csc[["UNITID","PCIP41_desc"]].groupby("PCIP41_desc").count()
# pcip_csc[["UNITID","PCIP42_desc"]].groupby("PCIP42_desc").count()
# pcip_csc[["UNITID","PCIP43_desc"]].groupby("PCIP43_desc").count()
# pcip_csc[["UNITID","PCIP44_desc"]].groupby("PCIP44_desc").count()
# pcip_csc[["UNITID","PCIP45_desc"]].groupby("PCIP45_desc").count()
# pcip_csc[["UNITID","PCIP46_desc"]].groupby("PCIP46_desc").count()
# pcip_csc[["UNITID","PCIP47_desc"]].groupby("PCIP47_desc").count()
# pcip_csc[["UNITID","PCIP48_desc"]].groupby("PCIP48_desc").count()
# pcip_csc[["UNITID","PCIP49_desc"]].groupby("PCIP49_desc").count()
# pcip_csc[["UNITID","PCIP50_desc"]].groupby("PCIP50_desc").count()
# pcip_csc[["UNITID","PCIP51_desc"]].groupby("PCIP51_desc").count()
# pcip_csc[["UNITID","PCIP52_desc"]].groupby("PCIP52_desc").count()
# pcip_csc[["UNITID","PCIP54_desc"]].groupby("PCIP54_desc").count()

# pcip_csc[["UNITID","PCIP01"]].groupby("PCIP01").count()
# pcip_csc[["UNITID","PCIP03"]].groupby("PCIP03").count()
# pcip_csc[["UNITID","PCIP04"]].groupby("PCIP04").count()
# pcip_csc[["UNITID","PCIP05"]].groupby("PCIP05").count()
# pcip_csc[["UNITID","PCIP09"]].groupby("PCIP09").count()
# pcip_csc[["UNITID","PCIP10"]].groupby("PCIP10").count()
# pcip_csc[["UNITID","PCIP11"]].groupby("PCIP11").count()
# pcip_csc[["UNITID","PCIP12"]].groupby("PCIP12").count()
# pcip_csc[["UNITID","PCIP13"]].groupby("PCIP13").count()
# pcip_csc[["UNITID","PCIP14"]].groupby("PCIP14").count()
# pcip_csc[["UNITID","PCIP15"]].groupby("PCIP15").count()
# pcip_csc[["UNITID","PCIP16"]].groupby("PCIP16").count()
# pcip_csc[["UNITID","PCIP19"]].groupby("PCIP19").count()
# pcip_csc[["UNITID","PCIP22"]].groupby("PCIP22").count()
# pcip_csc[["UNITID","PCIP23"]].groupby("PCIP23").count()
# pcip_csc[["UNITID","PCIP24"]].groupby("PCIP24").count()
# pcip_csc[["UNITID","PCIP25"]].groupby("PCIP25").count()
# pcip_csc[["UNITID","PCIP26"]].groupby("PCIP26").count()
# pcip_csc[["UNITID","PCIP27"]].groupby("PCIP27").count()
# pcip_csc[["UNITID","PCIP30"]].groupby("PCIP30").count()
# pcip_csc[["UNITID","PCIP31"]].groupby("PCIP31").count()
# pcip_csc[["UNITID","PCIP38"]].groupby("PCIP38").count()
# pcip_csc[["UNITID","PCIP39"]].groupby("PCIP39").count()
# pcip_csc[["UNITID","PCIP40"]].groupby("PCIP40").count()
# pcip_csc[["UNITID","PCIP41"]].groupby("PCIP41").count()
# pcip_csc[["UNITID","PCIP42"]].groupby("PCIP42").count()
# pcip_csc[["UNITID","PCIP43"]].groupby("PCIP43").count()
# pcip_csc[["UNITID","PCIP44"]].groupby("PCIP44").count()
# pcip_csc[["UNITID","PCIP45"]].groupby("PCIP45").count()
# pcip_csc[["UNITID","PCIP46"]].groupby("PCIP46").count()
# pcip_csc[["UNITID","PCIP47"]].groupby("PCIP47").count()
# pcip_csc[["UNITID","PCIP48"]].groupby("PCIP48").count()
# pcip_csc[["UNITID","PCIP49"]].groupby("PCIP49").count()
# pcip_csc[["UNITID","PCIP50"]].groupby("PCIP50").count()
# pcip_csc[["UNITID","PCIP51"]].groupby("PCIP51").count()
# pcip_csc[["UNITID","PCIP52"]].groupby("PCIP52").count()
# pcip_csc[["UNITID","PCIP54"]].groupby("PCIP54").count()

In [ ]:
# used while working but was not accurate

# pd.cut(pcip_csc['PCIP01'],                                  bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],
# labels = [                                  bins , "12.5%", "32.5%", "52.5%", "72.5%", "92.5%", "100"])

# pcip_csc["PCIP01_desc"] = pd.cut(pcip_csc["PCIP01"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["12.5%","32.5%","52.5%","72.5%","92.5%","100%"])
# pcip_csc["PCIP03_desc"] = pd.cut(pcip_csc["PCIP03"],bins = [0, 0.007, 0.0175, 0.028, 0.0385, 0.049, 1],labels = ["0.7%","1.75%","2.8%","3.85%","4.9%","100%"])
# pcip_csc["PCIP04_desc"] = pd.cut(pcip_csc["PCIP04"],bins = [0, 0.005, 0.0125, 0.02, 0.0275, 0.035, 1],labels = ["0.5%","1.25%","2%","2.75%","3.5%","100%"])
# pcip_csc["PCIP05_desc"] = pd.cut(pcip_csc["PCIP05"],bins = [0, 0.002, 0.0035, 0.005, 0.0065, 0.008, 1],labels = ["0.2%","0.35%","0.5%","0.65%","0.8%","100%"])
# pcip_csc["PCIP09_desc"] = pd.cut(pcip_csc["PCIP09"],bins = [0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 1],labels = ["2%","4%","6%","8%","10%","12%","100%"])
# pcip_csc["PCIP10_desc"] = pd.cut(pcip_csc["PCIP10"],bins = [0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004, 1],labels = ["0.11%","0.18%","0.24%","0.29%","0.35%","0.4%","100%"])
# pcip_csc["PCIP11_desc"] = pd.cut(pcip_csc["PCIP11"],bins = [0, 0.03, 0.05, 0.07, 0.09, 0.11, 1],labels = ["3%","5%","7%","9%","11%","100%"])
# pcip_csc["PCIP12_desc"] = pd.cut(pcip_csc["PCIP12"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1],labels = ["0.05%","2.05%","4.05%","6.05%","8.05%","100%"])
# pcip_csc["PCIP13_desc"] = pd.cut(pcip_csc["PCIP13"],bins = [0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475, 1],labels = ["0.1%","0.16%","0.22%","0.29%","0.35%","100%"])
# pcip_csc["PCIP14_desc"] = pd.cut(pcip_csc["PCIP14"],bins = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1],labels = ["3.8%","7.6%","11.4%","15.2%","19%","100%"])
# pcip_csc["PCIP15_desc"] = pd.cut(pcip_csc["PCIP15"],bins = [0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1],labels = ["3.8%","7.6%","11.4%","15.2%","19%","100%"])
# pcip_csc["PCIP16_desc"] = pd.cut(pcip_csc["PCIP16"],bins = [0, 0.001, 0.004, 0.007, 0.01, 1],labels = ["0.1%","0.4%","0.7%","1%","100%"])
# pcip_csc["PCIP19_desc"] = pd.cut(pcip_csc["PCIP19"],bins = [0, 0.005, 0.035, 0.065, 0.095, 1],labels = ["0.5%","3.5%","6.5%","9.5%","100%"])
# pcip_csc["PCIP22_desc"] = pd.cut(pcip_csc["PCIP22"],bins = [0, 0.01, 0.035, 0.065, 0.095, 1],labels = ["1%","3.5%","6.5%","9.5%","100%"])
# pcip_csc["PCIP23_desc"] = pd.cut(pcip_csc["PCIP23"],bins = [0, 0.001, 0.012, 0.023, 0.034, 1],labels = ["0.1%","1.2%","2.3%","3.4%","100%"])
# pcip_csc["PCIP24_desc"] = pd.cut(pcip_csc["PCIP24"],bins = [0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481, 1],labels = ["0.1%","1.21%","2.41%","3.61%","4.81%","100%"])
# pcip_csc["PCIP25_desc"] = pd.cut(pcip_csc["PCIP25"],bins = [0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135, 1],labels = ["0.1%","0.35%","0.6%","0.85%","1.1%","1.35%","100%"])
# pcip_csc["PCIP26_desc"] = pd.cut(pcip_csc["PCIP26"],bins = [0, 0.01, 0.04, 0.07, 0.1, 0.13, 1],labels = ["1%","4%","7%","10%","13%","100%"])
# pcip_csc["PCIP27_desc"] = pd.cut(pcip_csc["PCIP27"],bins = [0, 0.0025, 0.01, 0.0175, 0.025, 0.0325, 1],labels = ["0.25%","1%","1.75%","2.5%","3.25%","100%"])
# pcip_csc["PCIP30_desc"] = pd.cut(pcip_csc["PCIP30"],bins = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1],labels = ["0.1%","1%","1.9%","2.8%","3.7%","4.6%","100%"])
# pcip_csc["PCIP31_desc"] = pd.cut(pcip_csc["PCIP31"],bins = [0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1],labels = ["0.1%","1%","1.9%","2.8%","3.7%","4.6%","100%"])
# pcip_csc["PCIP38_desc"] = pd.cut(pcip_csc["PCIP38"],bins = [0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036, 1],labels = ["0.1%","0.8%","1.5%","2.2%","2.9%","3.6%","100%"])
# pcip_csc["PCIP39_desc"] = pd.cut(pcip_csc["PCIP39"],bins = [0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275, 1],labels = ["0.08%","0.63%","1.18%","1.73%","2.28%","100%"])
# pcip_csc["PCIP40_desc"] = pd.cut(pcip_csc["PCIP40"],bins = [0, 0.001, 0.006, 0.011, 0.016, 0.021, 1],labels = ["0.1%","0.6%","1.1%","1.6%","2.1%","100%"])
# pcip_csc["PCIP41_desc"] = pd.cut(pcip_csc["PCIP41"],bins = [0, 0.0005, 0.003, 0.0055, 0.008, 0.0105, 1],labels = ["0.05%","0.3%","0.55%","0.8%","1.05%","100%"])
# pcip_csc["PCIP42_desc"] = pd.cut(pcip_csc["PCIP42"],bins = [0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11, 1],labels = ["3.5%","4.75%","6%","7.25%","8.5%","9.75%","11%","100%"])
# pcip_csc["PCIP43_desc"] = pd.cut(pcip_csc["PCIP43"],bins = [0, 0.01, 0.03, 0.05, 0.07, 0.09, 1],labels = ["1%","3%","5%","7%","9%","100%"])
# pcip_csc["PCIP44_desc"] = pd.cut(pcip_csc["PCIP44"],bins = [0, 0.005, 0.02, 0.035, 0.05, 0.065, 1],labels = ["0.5%","2%","3.5%","5%","6.5%","100%"])
# pcip_csc["PCIP45_desc"] = pd.cut(pcip_csc["PCIP45"],bins = [0, 0.005, 0.02, 0.035, 0.05, 0.065, 1],labels = ["0.5%","2%","3.5%","5%","6.5%","100%"])
# pcip_csc["PCIP46_desc"] = pd.cut(pcip_csc["PCIP46"],bins = [0, 0.0103, 0.037796, 0.125, 1],labels = ["1.03%","3.78%","12.5%","100%"])
# pcip_csc["PCIP47_desc"] = pd.cut(pcip_csc["PCIP47"],bins = [0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615, 1],labels = ["0.05%","1.65%","3.15%","4.65%","6.15%","100%"])
# pcip_csc["PCIP48_desc"] = pd.cut(pcip_csc["PCIP48"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1],labels = ["0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"])
# pcip_csc["PCIP49_desc"] = pd.cut(pcip_csc["PCIP49"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1],labels = ["0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"])
# pcip_csc["PCIP50_desc"] = pd.cut(pcip_csc["PCIP50"],bins = [0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1],labels = ["0.05%","2.05%","4.05%","6.05%","8.05%","100%"])
# pcip_csc["PCIP51_desc"] = pd.cut(pcip_csc["PCIP51"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["12.5%","32.5%","52.5%","72.5%","92.5%","100%"])
# pcip_csc["PCIP52_desc"] = pd.cut(pcip_csc["PCIP52"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["12.5%","32.5%","52.5%","72.5%","92.5%","100%"])
# pcip_csc["PCIP54_desc"] = pd.cut(pcip_csc["PCIP54"],bins = [0, 0.125, 0.325, 0.525, 0.725, 0.925, 1],labels = ["12.5%","32.5%","52.5%","72.5%","92.5%","100%"])

# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["0%","12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.007, 0.0175, 0.028, 0.0385, 0.049, 1]).shape,pd.DataFrame(["0%","0.7%","1.75%","2.8%","3.85%","4.9%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.0125, 0.02, 0.0275, 0.035, 1]).shape,pd.DataFrame(["0%","0.5%","1.25%","2%","2.75%","3.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.002, 0.0035, 0.005, 0.0065, 0.008, 1]).shape,pd.DataFrame(["0%","0.2%","0.35%","0.5%","0.65%","0.8%","100%"]).shape)
# print(pd.DataFrame([0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 1]).shape,pd.DataFrame(["0%","2%","4%","6%","8%","10%","12%","100%"]).shape)
# print(pd.DataFrame([0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004, 1]).shape,pd.DataFrame(["0%","0.11%","0.18%","0.24%","0.29%","0.35%","0.4%","100%"]).shape)
# print(pd.DataFrame([0, 0.03, 0.05, 0.07, 0.09, 0.11, 1]).shape,pd.DataFrame(["0%","3%","5%","7%","9%","11%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1]).shape,pd.DataFrame(["0%","0.05%","2.05%","4.05%","6.05%","8.05%","100%"]).shape)
# print(pd.DataFrame([0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475, 1]).shape,pd.DataFrame(["0%","0.1%","0.16%","0.22%","0.29%","0.35%","100%"]).shape)
# print(pd.DataFrame([0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1]).shape,pd.DataFrame(["0%","3.8%","7.6%","11.4%","15.2%","19%","100%"]).shape)
# print(pd.DataFrame([0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1]).shape,pd.DataFrame(["0%","3.8%","7.6%","11.4%","15.2%","19%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.004, 0.007, 0.01, 1]).shape,pd.DataFrame(["0%","0.1%","0.4%","0.7%","1%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.035, 0.065, 0.095, 1]).shape,pd.DataFrame(["0%","0.5%","3.5%","6.5%","9.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.01, 0.035, 0.065, 0.095, 1]).shape,pd.DataFrame(["0%","1%","3.5%","6.5%","9.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.012, 0.023, 0.034, 1]).shape,pd.DataFrame(["0%","0.1%","1.2%","2.3%","3.4%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481, 1]).shape,pd.DataFrame(["0%","0.1%","1.21%","2.41%","3.61%","4.81%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135, 1]).shape,pd.DataFrame(["0%","0.1%","0.35%","0.6%","0.85%","1.1%","1.35%","100%"]).shape)
# print(pd.DataFrame([0, 0.01, 0.04, 0.07, 0.1, 0.13, 1]).shape,pd.DataFrame(["0%","1%","4%","7%","10%","13%","100%"]).shape)
# print(pd.DataFrame([0, 0.0025, 0.01, 0.0175, 0.025, 0.0325, 1]).shape,pd.DataFrame(["0%","0.25%","1%","1.75%","2.5%","3.25%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1]).shape,pd.DataFrame(["0%","0.1%","1%","1.9%","2.8%","3.7%","4.6%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1]).shape,pd.DataFrame(["0%","0.1%","1%","1.9%","2.8%","3.7%","4.6%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036, 1]).shape,pd.DataFrame(["0%","0.1%","0.8%","1.5%","2.2%","2.9%","3.6%","100%"]).shape)
# print(pd.DataFrame([0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275, 1]).shape,pd.DataFrame(["0%","0.08%","0.63%","1.18%","1.73%","2.28%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.006, 0.011, 0.016, 0.021, 1]).shape,pd.DataFrame(["0%","0.1%","0.6%","1.1%","1.6%","2.1%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.003, 0.0055, 0.008, 0.0105, 1]).shape,pd.DataFrame(["0%","0.05%","0.3%","0.55%","0.8%","1.05%","100%"]).shape)
# print(pd.DataFrame([0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11, 1]).shape,pd.DataFrame(["0%","3.5%","4.75%","6%","7.25%","8.5%","9.75%","11%","100%"]).shape)
# print(pd.DataFrame([0, 0.01, 0.03, 0.05, 0.07, 0.09, 1]).shape,pd.DataFrame(["0%","1%","3%","5%","7%","9%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.02, 0.035, 0.05, 0.065, 1]).shape,pd.DataFrame(["0%","0.5%","2%","3.5%","5%","6.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.02, 0.035, 0.05, 0.065, 1]).shape,pd.DataFrame(["0%","0.5%","2%","3.5%","5%","6.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.0103, 0.037796, 0.125, 1]).shape,pd.DataFrame(["0%","1.03%","3.78%","12.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615, 1]).shape,pd.DataFrame(["0%","0.05%","1.65%","3.15%","4.65%","6.15%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1]).shape,pd.DataFrame(["0%","0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1]).shape,pd.DataFrame(["0%","0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1]).shape,pd.DataFrame(["0%","0.05%","2.05%","4.05%","6.05%","8.05%","100%"]).shape)
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["0%","12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["0%","12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["0%","12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)

# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.007, 0.0175, 0.028, 0.0385, 0.049, 1]).shape,pd.DataFrame(["0.7%","1.75%","2.8%","3.85%","4.9%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.0125, 0.02, 0.0275, 0.035, 1]).shape,pd.DataFrame(["0.5%","1.25%","2%","2.75%","3.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.002, 0.0035, 0.005, 0.0065, 0.008, 1]).shape,pd.DataFrame(["0.2%","0.35%","0.5%","0.65%","0.8%","100%"]).shape)
# print(pd.DataFrame([0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 1]).shape,pd.DataFrame(["2%","4%","6%","8%","10%","12%","100%"]).shape)
# print(pd.DataFrame([0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004, 1]).shape,pd.DataFrame(["0.11%","0.18%","0.24%","0.29%","0.35%","0.4%","100%"]).shape)
# print(pd.DataFrame([0, 0.03, 0.05, 0.07, 0.09, 0.11, 1]).shape,pd.DataFrame(["3%","5%","7%","9%","11%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1]).shape,pd.DataFrame(["0.05%","2.05%","4.05%","6.05%","8.05%","100%"]).shape)
# print(pd.DataFrame([0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475, 1]).shape,pd.DataFrame(["0.1%","0.16%","0.22%","0.29%","0.35%","100%"]).shape)
# print(pd.DataFrame([0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1]).shape,pd.DataFrame(["3.8%","7.6%","11.4%","15.2%","19%","100%"]).shape)
# print(pd.DataFrame([0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1]).shape,pd.DataFrame(["3.8%","7.6%","11.4%","15.2%","19%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.004, 0.007, 0.01, 1]).shape,pd.DataFrame(["0.1%","0.4%","0.7%","1%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.035, 0.065, 0.095, 1]).shape,pd.DataFrame(["0.5%","3.5%","6.5%","9.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.01, 0.035, 0.065, 0.095, 1]).shape,pd.DataFrame(["1%","3.5%","6.5%","9.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.012, 0.023, 0.034, 1]).shape,pd.DataFrame(["0.1%","1.2%","2.3%","3.4%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481, 1]).shape,pd.DataFrame(["0.1%","1.21%","2.41%","3.61%","4.81%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135, 1]).shape,pd.DataFrame(["0.1%","0.35%","0.6%","0.85%","1.1%","1.35%","100%"]).shape)
# print(pd.DataFrame([0, 0.01, 0.04, 0.07, 0.1, 0.13, 1]).shape,pd.DataFrame(["1%","4%","7%","10%","13%","100%"]).shape)
# print(pd.DataFrame([0, 0.0025, 0.01, 0.0175, 0.025, 0.0325, 1]).shape,pd.DataFrame(["0.25%","1%","1.75%","2.5%","3.25%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1]).shape,pd.DataFrame(["0.1%","1%","1.9%","2.8%","3.7%","4.6%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1]).shape,pd.DataFrame(["0.1%","1%","1.9%","2.8%","3.7%","4.6%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036, 1]).shape,pd.DataFrame(["0.1%","0.8%","1.5%","2.2%","2.9%","3.6%","100%"]).shape)
# print(pd.DataFrame([0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275, 1]).shape,pd.DataFrame(["0.08%","0.63%","1.18%","1.73%","2.28%","100%"]).shape)
# print(pd.DataFrame([0, 0.001, 0.006, 0.011, 0.016, 0.021, 1]).shape,pd.DataFrame(["0.1%","0.6%","1.1%","1.6%","2.1%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.003, 0.0055, 0.008, 0.0105, 1]).shape,pd.DataFrame(["0.05%","0.3%","0.55%","0.8%","1.05%","100%"]).shape)
# print(pd.DataFrame([0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11, 1]).shape,pd.DataFrame(["3.5%","4.75%","6%","7.25%","8.5%","9.75%","11%","100%"]).shape)
# print(pd.DataFrame([0, 0.01, 0.03, 0.05, 0.07, 0.09, 1]).shape,pd.DataFrame(["1%","3%","5%","7%","9%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.02, 0.035, 0.05, 0.065, 1]).shape,pd.DataFrame(["0.5%","2%","3.5%","5%","6.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.005, 0.02, 0.035, 0.05, 0.065, 1]).shape,pd.DataFrame(["0.5%","2%","3.5%","5%","6.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.0103, 0.037796, 0.125, 1]).shape,pd.DataFrame(["1.03%","3.78%","12.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615, 1]).shape,pd.DataFrame(["0.05%","1.65%","3.15%","4.65%","6.15%","100%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1]).shape,pd.DataFrame(["0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1]).shape,pd.DataFrame(["0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"]).shape)
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1]).shape,pd.DataFrame(["0.05%","2.05%","4.05%","6.05%","8.05%","100%"]).shape)
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape,pd.DataFrame(["12.5%","32.5%","52.5%","72.5%","92.5%","100%"]).shape)

# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape[0] - pd.DataFrame(["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"]).shape[0])
# print(pd.DataFrame([0, 0.007, 0.0175, 0.028, 0.0385, 0.049, 1]).shape[0] - pd.DataFrame(["0%", "0.7%","1.75%","2.8%","3.85%","4.9%"]).shape[0])
# print(pd.DataFrame([0, 0.005, 0.0125, 0.02, 0.0275, 0.035, 1]).shape[0] - pd.DataFrame(["0%", "0.5%","1.25%","2%","2.75%","3.5%"]).shape[0])
# print(pd.DataFrame([0, 0.002, 0.0035, 0.005, 0.0065, 0.008, 1]).shape[0] - pd.DataFrame(["0%", "0.2%","0.35%","0.5%","0.65%","0.8%"]).shape[0])
# print(pd.DataFrame([0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 1]).shape[0] - pd.DataFrame(["0%", "2%","4%","6%","8%","10%","12%"]).shape[0])
# print(pd.DataFrame([0, 0.0011, 0.0018, 0.00235, 0.0029, 0.00345, 0.004, 1]).shape[0] - pd.DataFrame(["0%", "0.11%","0.18%","0.24%","0.29%","0.35%","0.4%"]).shape[0])
# print(pd.DataFrame([0, 0.03, 0.05, 0.07, 0.09, 0.11, 1]).shape[0] - pd.DataFrame(["0%", "3%","5%","7%","9%","11%"]).shape[0])
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1]).shape[0] - pd.DataFrame(["0%", "0.05%","2.05%","4.05%","6.05%","8.05%"]).shape[0])
# print(pd.DataFrame([0, 0.0010, 0.001625, 0.002225, 0.00285, 0.003475, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","0.16%","0.22%","0.29%","0.35%"]).shape[0])
# print(pd.DataFrame([0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1]).shape[0] - pd.DataFrame(["0%", "3.8%","7.6%","11.4%","15.2%","19%", "100%"]).shape[0])
# print(pd.DataFrame([0, 0.00001, 0.03801, 0.07601, 0.11401, 0.15201, 0.19001, 1]).shape[0] - pd.DataFrame(["0%", "3.8%","7.6%","11.4%","15.2%","19%", "100%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.004, 0.007, 0.01, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","0.4%","0.7%","1%"]).shape[0])
# print(pd.DataFrame([0, 0.005, 0.035, 0.065, 0.095, 1]).shape[0] - pd.DataFrame(["0%", "0.5%","3.5%","6.5%","9.5%"]).shape[0])
# print(pd.DataFrame([0, 0.01, 0.035, 0.065, 0.095, 1]).shape[0] - pd.DataFrame(["0%", "1%","3.5%","6.5%","9.5%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.012, 0.023, 0.034, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","1.2%","2.3%","3.4%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.0121, 0.0241, 0.0361, 0.0481, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","1.21%","2.41%","3.61%","4.81%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.0035, 0.006, 0.0085, 0.011, 0.0135, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","0.35%","0.6%","0.85%","1.1%","1.35%"]).shape[0])
# print(pd.DataFrame([0, 0.01, 0.04, 0.07, 0.1, 0.13, 1]).shape[0] - pd.DataFrame(["0%", "1%","4%","7%","10%","13%"]).shape[0])
# print(pd.DataFrame([0, 0.0025, 0.01, 0.0175, 0.025, 0.0325, 1]).shape[0] - pd.DataFrame(["0%", "0.25%","1%","1.75%","2.5%","3.25%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","1%","1.9%","2.8%","3.7%","4.6%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.01, 0.019, 0.028, 0.037, 0.046, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","1%","1.9%","2.8%","3.7%","4.6%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.008, 0.015, 0.022, 0.029, 0.036, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","0.8%","1.5%","2.2%","2.9%","3.6%"]).shape[0])
# print(pd.DataFrame([0, 0.00075, 0.00625, 0.01175, 0.01725, 0.02275, 1]).shape[0] - pd.DataFrame(["0%", "0.08%","0.63%","1.18%","1.73%","2.28%"]).shape[0])
# print(pd.DataFrame([0, 0.001, 0.006, 0.011, 0.016, 0.021, 1]).shape[0] - pd.DataFrame(["0%", "0.1%","0.6%","1.1%","1.6%","2.1%"]).shape[0])
# print(pd.DataFrame([0, 0.0005, 0.003, 0.0055, 0.008, 0.0105, 1]).shape[0] - pd.DataFrame(["0%", "0.05%","0.3%","0.55%","0.8%","1.05%"]).shape[0])
# print(pd.DataFrame([0, 0.035, 0.0475, 0.06, 0.0725, 0.085, 0.0975, 0.11, 1]).shape[0] - pd.DataFrame(["0%", "3.5%","4.75%","6%","7.25%","8.5%","9.75%","11%"]).shape[0])
# print(pd.DataFrame([0, 0.01, 0.03, 0.05, 0.07, 0.09, 1]).shape[0] - pd.DataFrame(["0%", "1%","3%","5%","7%","9%"]).shape[0])
# print(pd.DataFrame([0, 0.005, 0.02, 0.035, 0.05, 0.065, 1]).shape[0] - pd.DataFrame(["0%", "0.5%","2%","3.5%","5%","6.5%"]).shape[0])
# print(pd.DataFrame([0, 0.005, 0.02, 0.035, 0.05, 0.065, 1]).shape[0] - pd.DataFrame(["0%", "0.5%","2%","3.5%","5%","6.5%"]).shape[0])
# print(pd.DataFrame([0, 0.0103, 0.037796, 0.125, 1]).shape[0] - pd.DataFrame(["0%", "1.03%","3.78%","12.5%"]).shape[0])
# print(pd.DataFrame([0, 0.0005, 0.0165, 0.0315, 0.0465, 0.0615, 1]).shape[0] - pd.DataFrame(["0%", "0.05%","1.65%","3.15%","4.65%","6.15%"]).shape[0])
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1]).shape[0] - pd.DataFrame(["0%", "0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"]).shape[0])
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 0.1005, 1]).shape[0] - pd.DataFrame(["0%", "0.05%","2.05%","4.05%","6.05%","8.05%","10.05%"]).shape[0])
# print(pd.DataFrame([0, 0.0005, 0.0205, 0.0405, 0.0605, 0.0805, 1]).shape[0] - pd.DataFrame(["0%", "0.05%","2.05%","4.05%","6.05%","8.05%"]).shape[0])
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape[0] - pd.DataFrame(["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"]).shape[0])
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape[0] - pd.DataFrame(["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"]).shape[0])
# print(pd.DataFrame([0, 0.125, 0.325, 0.525, 0.725, 0.925, 1]).shape[0] - pd.DataFrame(["0%", "12.5%","32.5%","52.5%","72.5%","92.5%"]).shape[0])